1. Key Entities & Relationships
Teams: Each team belongs to a grade and has constraints on when they can or cannot play.
Club: Teams belong to a club. There may be constraints on when a whole club may want to play.
Grades: A grouping of teams where teams play each other in a round-robin format.
Games: Scheduled matchups between teams at a specific time and date.
Fields: The fields that can be used for match ups.
Week: A roster of all games to be played in that week.
Roster: A complete schedule that assigns teams to timeslots while respecting constraints.
Timeslots: A complete calendar of timeslots that are available for use in the roster.

2. Constraints & Optimization Goals
We need to ensure:

Hard Constraints (Must Be Enforced)

Teams cant play more than one a game a week
Each field cannot have more than on game per timeslot
Each team in each grade must play the same number of games
Each team in each grade must play exactly each other team the same amount of times. Slight flexibility lower grades, PHL 2nds must adhere.
PHL between 11:30 and 4:30
Teams must only play other teams in the same grade. 
Each team appears once per round (or gets a bye).
No team is scheduled for two games at the same time.
Games must be assigned to valid time slots.
Ensure that only home team games are held outside Newcastle International Hockey Centre
Staggered phl games, no two phl games at the same time, but only at Newcastle International Hockey Centre
A team should not have two byes in a row.



Soft Constraints (Optimization Goals):
PHL and 2nds cannot play at the same time or at different field locations concurrently

When club plays away at maitland, try to get all of that clubs games vs maitland up at maitland 
When maitland plays away, try to get all of maitlands games away (note this is at location Newcastle International Hockey Centre)
When maitland in maitland, have games in order of grade, so lowest grade plays at the earliest time slot and highest grade plays at the latest time slot
Club sequencing, try to group club games in a day so that theyre back to back
minimise games that are played later in the day on sunday
Maitland should play 50% of games at home 


Soft Contraints provided by Clubs:
Club day
Club day: is sequence and on the same field, and specify day too, ifmultiple teams in a grade, play each other on club day
Cannot play condition at specific date/timeslot
teams cannot play at the same time, not back to back timeslots at different fields
hopeful not play slot 

in timeslots, there will be unavailable times, but also weekends with not a full round but can be used for make up games
first iteration is for only first 4 weeks as there may be new teams etc and changes

Teams should be scheduled to play at their preferred times when possible.
Balanced matchups: Teams should play each other an even number of times over the competition.
Even distribution of byes (if a grade has an odd number of teams).






# To Do
Generate txt document of hard, soft constraints, broken by permanent or maybe change

    Add in round number to game pairings
    Enforce back to back byes based on round number
    Round number is to be the availabe week number in broadmeadow, games held in maitland or wyong will probably be previous round if no broadmeadow games
    round number may not equal number of games played by teams
    so num_rounds becomes a grade based dictionary
    round number must match number of primary weeks at broadmeadow

    Change maitland50/50 to home and away 50/50, make a hard constraint and ensure that the dicts are broken down by team pairings

    Edit and game adjacency to be based on times not day_slot, as day slots will change across fields, 
    Edit day time maps to include fields (wyong only 3pm Sunday, or Friday 8pm)

    Teams do not play the same team within any 3 week period

    Get a constraint weight per soft constraint so can adjust the weight to match

    Soft constraint to maximise the number of teams per timeslot at broadmeadow, this way no club plays at the same time and it enforces no field gaps
    Simultaneously, for each field, minimise the number of teams playing on it

PHL time slot at broadmeadow is 7pm Fridays
PHL Broadmeadow dates are 20/06, 30/05, 1/08
20/06 Tigers vs Wests 
30/05 Souths vs Maitland
1/08 Tigers vs Someone

Write code that imports current fixed weeks, and sets fixed games

    Change friday ngiht matches to only line up with clubs that have juniors

Make club day so that a) all teams are forced to play and b) that if there are multiple in a grade, force that pairing to occur

Change clubs to naming convention as provided by magix, as well as update to look like what John Fernance needs

    Edit BACK TO BACK BYES to skip 6th grade

    check out day_slot assignation, wyong 3pm game gets dayslot 5 but it is the only one therre

look at minimiselategames, club day, and grade order to reduce constraints

investigate phl preferences having 0 weight

edit bye and 2nd grade adjacency checks 




# Remaining To Do
Currently phl preferred times not checked for in analysis. You should rework both into a condition where you specify game details, and a prefer or not prefer, can wind everything into one easily then
BIG ONE, YOU DID NOT ENFORCE GRADE ORDER WITH CLUB DAYS

# Notes
Send draw to Gosford and Maitland the moment it is done
make 6th grade have 21 rounds

All Wyong games to be hosted at 3pm Sundays on Sundays
shift maitland times 30 minutes forward
# Plan
Email Michael Wallis about Friday night games
check if phl can play on south field
email rolls with info

write up cleaner analysis
write up code that ensures keys of dicts match known clubs and teams, this starts to belong to tidy and foolproof


# Considerations to Investigate/ Code still
Port vs Maitland days


# PHL
try not to put tigers vs norths round 1
Maitland, ok with 8pm games on friday in Newcastle International Hockey Centre or wyong PHL
Please be advised Tigers are happy to play there 2 central coast games on Friday nights, 8pm slot
Max 3 friday night games for phl, Newcastle International Hockey Centre only
Friday phl timeslot 8pm
Port plays at Maitland we tend to have all our mens teams and our womens teams from the Maitland comp play each other.

In [ ]:
from pydantic import BaseModel, Field,  field_validator
from typing import List, Dict, Tuple
from datetime import time as tm
from datetime import timedelta, datetime
from ortools.sat.python import cp_model
import pandas as pd
import os
from collections import defaultdict
import re
import json
from abc import ABC, abstractmethod
import math
import numpy as np
from itertools import chain

class PlayingField(BaseModel):
    name: str = Field(..., description="Name of the field")
    location: str = Field(..., description="Location of the field")

    def __getattr__(self, attr):
        if attr == "field_location":
            return self.location
        elif attr == "field_name":
            return self.name
        raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{attr}'")
    
class Grade(BaseModel):
    name: str = Field(..., description="Grade name")
    teams: List[str] = Field(..., description="List of team names in this grade")
    num_teams: int = Field(0, description="Number of teams in this grade")


    def __init__(self, **data):
        super().__init__(**data)
        object.__setattr__(self, "num_teams", len(self.teams))

    def __lt__(self, other):
        expected_order = ["PHL", "2nd", "3rd", "4th", "5th", "6th"]  

        if not isinstance(other, Grade):
            return NotImplemented
        try:
            self_index = expected_order.index(self.name)
            other_index = expected_order.index(other.name)
        except ValueError:
            raise ValueError(f"Unknown grade in comparison: {self.name} or {other.name}")

        return self_index > other_index    
    
class Timeslot(BaseModel):
    date: str = Field(..., description="Date of the game (e.g., '2025-03-04')")
    day: str = Field(..., description="Day of the game (e.g., 'Saturday', 'Sunday')")
    time: str = Field(..., description="Time of the game (e.g., 14:00 for 2 PM)")
    week: int = Field(..., description="The week number for the season")
    day_slot: int = Field(..., description="The game slot for the day (e.g., 1 for first game of the day)")
    field: PlayingField = Field(..., description="Field where the game is played")
    round_no: int = Field(..., description="Round number for the season")

class Club(BaseModel):
    name: str = Field(..., description="Club name")
    home_field: str = Field(..., description="Home field")
    preferred_times: List[Timeslot] = Field(default=[], description="Preferred play times for the club")
    num_teams: int = Field(0, description="Number of teams in this club")

class Team(BaseModel):
    name: str = Field(..., description="Name of the team")
    club: Club = Field(..., description="Club the team belongs to")
    grade: str = Field(..., description="Grade the team belongs to")
    preferred_times: List[Timeslot] = Field(default=[], description="Times the team prefers to play")
    unavailable_times: List[Timeslot] = Field(default=[], description="Times the team cannot play")
    constraints: List[str] = Field(default=[], description="Special scheduling constraints for the team")

class ClubDay(BaseModel):
    date: str = Field(..., description="Date of the game (e.g., '2025-03-04')")
    day: str = Field(..., description="Day of the game (e.g., 'Saturday', 'Sunday')")
    week: int = Field(..., description="The week number for the season")
    field: PlayingField = Field(..., description="Field where the game is played")


class Game(BaseModel):
    team1: str = Field(..., description="First team playing")
    team2: str = Field(..., description="Second team playing")
    timeslot: Timeslot = Field(..., description="Scheduled time for the game")
    field: PlayingField = Field(..., description="Field where the game is played")
    grade: Grade = Field(..., description="Grade the game belongs to")

class WeeklyDraw(BaseModel):
    week: int = Field(..., description="Week number in the season")
    round_no: int = Field(..., description="Round number for the season")
    games: List[Game] = Field(..., description="Games scheduled for this week")
    bye_teams: List[str] = Field(default=[], description="Teams with a bye this week")
    
class Roster(BaseModel):
    weeks: List[WeeklyDraw] = Field(..., description="Complete schedule for the season")

FIELDS = [PlayingField(location='Newcastle International Hockey Centre', name='SF'),
          PlayingField(location='Newcastle International Hockey Centre', name='EF'),
          PlayingField(location='Newcastle International Hockey Centre', name='WF'),
          PlayingField(location='Maitland Park', name='Maitland Main Field',),
          PlayingField(location='Central Coast Hockey Park', name='Wyong Main Field'),]

TEAMS_DATA = r'C:\Users\c3205\Documents\Code\Jupyter Notebooks\draw\data\2025\teams'
CLUBS = []
TEAMS = []

for file in os.listdir(TEAMS_DATA):
    df = pd.read_csv(os.path.join(TEAMS_DATA, file))
    club = Club(name=df['Club'].iloc[0], home_field="Maitland Park" if df['Club'].iloc[0] == "Maitland" else 
                      'Central Coast Hockey Park' if df['Club'].iloc[0] == 'Gosford' else "Newcastle International Hockey Centre")
    CLUBS.append(club)
    teams = [Team(name=f"{row['Team Name']} {row['Grade']}", club=club, grade=row['Grade'], home_field=club.home_field) for index, row in df.iterrows()]
    TEAMS.extend(teams)

teams_by_grade = defaultdict(list)
for team in TEAMS:
    teams_by_grade[team.grade].append(team.name)
GRADES = [Grade(name=f'{grade}', teams=teams) for grade, teams in sorted(teams_by_grade.items())]

teams_by_club = defaultdict(list)
for team in TEAMS:
    teams_by_club[team.club.name].append(team.name)

for club, teams in teams_by_club.items():
    club_obj = next((c for c in CLUBS if c.name == club), None)
    club_obj.num_teams = len(teams)
    
for grade in GRADES:
    print(grade.name, grade.teams, grade.num_teams)

for club in CLUBS:  
    print(club.name, club.num_teams)

2nd ['Maitland 2nd', 'Norths 2nd', 'Souths 2nd', 'Tigers 2nd', 'Wests 2nd'] 5
3rd ['Crusaders 3rd', 'Maitland 3rd', 'Norths 3rd', 'Port Stephens 3rd', 'Souths 3rd', 'Tigers 3rd', 'University  3rd', 'University Seapigs 3rd', 'Wests Red 3rd', 'Wests Green 3rd'] 10
4th ['Colts 4th', 'Maitland 4th', 'Norths 4th', 'Port Stephens 4th', 'Souths 4th', 'Tigers 4th', 'University 4th', 'Wests Red 4th', 'Wests Green 4th'] 9
5th ['Colts 5th', 'Crusaders 5th', 'Maitland 5th', 'Norths 5th', 'Port Stephens 5th', 'Tigers Yellow 5th', 'Tigers Black 5th', 'University Seapigs 5th', 'University  5th', 'Wests  5th'] 10
6th ['Colts 6th', 'Crusaders 6th', 'Maitland 6th', 'Souths 6th', 'Tigers 6th', 'University Gentlemen 6th', 'University 6th', 'Wests 6th'] 8
PHL ['Gosford PHL', 'Maitland PHL', 'Norths PHL', 'Souths PHL', 'Tigers PHL', 'Wests PHL'] 6
Colts 3
Crusaders 3
Gosford 1
Maitland 6
Norths 5
Port Stephens 3
Souths 5
Tigers 7
University 7
Wests 8


In [22]:
def get_club(team_name, teams):
    for team in teams:
        if team_name == team.name:
            return team.club.name
    raise ValueError(f"Team {team_name} not found in teams when calling get_club") 

def get_club_object(team_name, teams):
    for team in teams:
        if team_name == team.name:
            return team.club
    raise ValueError(f"Team {team_name} not found in teams when calling get_club_object") 

def get_teams_from_club(club_name, teams):
    return [team.name for team in teams if team.club.name == club_name]

def get_club_from_clubname(club, CLUBS):
    for c in CLUBS:
        if c.name == club:
            return c
    raise ValueError(f"Club {club} not found in CLUBS when calling get_club_from_clubname")

def get_duplicated_graded_teams(club, grade, teams):
    dup_teams = []
    for team in teams:
        if team.club.name == club and team.grade == grade:
            dup_teams.append(team.name)
    return dup_teams

def split_number_suffix(text):
    match = re.match(r"(\d+)([a-zA-Z]+)", text)  
    if match:
        return match.group(1), match.group(2)  
    
    return text, ""  

def add_ordinal_suffix(number):
    if 11 <= number % 100 <= 13: 
        suffix = "th"
    else:
        suffix = {1: "st", 2: "nd", 3: "rd"}.get(number % 10, "th")
    
    return f"{number}{suffix}"

class Constraint(ABC):
    """Abstract base class for all scheduling constraints."""
    
    @abstractmethod
    def apply(self, model: cp_model.CpModel, X: dict, data: dict):
        """Apply constraint to the OR-Tools model."""
        pass

class NoDoubleBookingTeamsConstraint(Constraint):
    """Ensure no team is scheduled for more than one game per week."""
    
    def apply(self, model, X, data):
        games = data['games']
        timeslots = data['timeslots']
        
        weekly_games = defaultdict(list)

        for t in timeslots:
            for (t1, t2, grade) in games:
                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                if key in X:
                    weekly_games[(t.week, t1)].append(X[key])
                    weekly_games[(t.week, t2)].append(X[key])

        for (week, team), game_vars in weekly_games.items():
            model.Add(sum(game_vars) <= 1)

class NoDoubleBookingFieldsConstraint(Constraint):
    """Ensure no field is scheduled for more than one game per time slot."""
    
    def apply(self, model, X, data):
        games = data['games']
        timeslots = data['timeslots']
        
        field_usage = defaultdict(list)

        for t in timeslots:
            for (t1, t2, grade) in games:
                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                if key in X:
                    field_usage[(t.day, t.day_slot, t.week, t.field.name)].append(X[key])

        for slot, game_vars in field_usage.items():
            model.Add(sum(game_vars) <= 1)

class EnsureEqualGamesAndBalanceMatchUps(Constraint):
    """Ensure each team plays the most it can."""
    
    def apply(self, model, X, data):
        games = data['games']
        timeslots = data['timeslots']
        num_rounds = data['num_rounds']

        team_games = defaultdict(lambda: defaultdict(list))
        grade_games = defaultdict(lambda: defaultdict(list))

        for t in timeslots:
            for (t1, t2, grade) in games:
                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                if key in X:
                    team_games[grade][t1].append(X[key])
                    team_games[grade][t2].append(X[key])
                    grade_games[grade][tuple(sorted((t1, t2)))].append(X[key])

        # Ensure that each team plays the correct number of games
        for grade, teams in team_games.items():
            num_teams = len([team for team in data['teams'] if team.grade == grade])
            total_games = (num_rounds[grade] // (num_teams - 1) ) * (num_teams - 1) if num_teams % 2 == 0 else (num_rounds[grade] //num_teams) * (num_teams - 1)

            print(f'Aim total games for grade {grade} is {total_games}.')

            for team, game_vars in teams.items():
                model.Add(sum(game_vars) == total_games)  

        # Ensure that each team plays each other team exactly the same amount of times
        for grade, teams in grade_games.items():
            num_teams = len([team for team in data['teams'] if team.grade == grade])
            total_games = (num_rounds[grade] // (num_teams - 1) )  if num_teams % 2 == 0 else (num_rounds[grade] //num_teams) 
            
            for team, game_vars in teams.items():
                model.Add(sum(game_vars) == total_games) 

class PHLAndSecondGradeAdjacency(Constraint):
    '''Ensure that PHL and 2nds do not play in adjacent day_slots at different locations.''' 

    def apply(self, model, X, data):
        games = data['games']
        timeslots = data['timeslots']

        phl_games = defaultdict(lambda: defaultdict(list))

        fields = data['fields']
        field_locations = [field.location for field in fields]
        max_day_slot_per_field = data['max_day_slot_per_field']

        for t in timeslots:
            for (t1, t2, grade) in games:
                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)

                if grade == 'PHL' and key in X:
                    club = get_club(t1, data['teams'])
                    dup_2nds = get_duplicated_graded_teams(club, '2nd', data['teams'])
                    for team in dup_2nds:
                        phl_games[(club, team, t.week, t.day)][(t.time, t.field.location)].append(X[key])


                    club = get_club(t2, data['teams'])
                    dup_2nds = get_duplicated_graded_teams(club, '2nd', data['teams'])

                    for team in dup_2nds:
                        phl_games[(club, team, t.week, t.day)][(t.time, t.field.location)].append(X[key])

        for t in timeslots:
            for (t1, t2, grade) in games:
                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)

                if grade == '2nd' and key in X:
                    club = get_club(t1, data['teams'])
                    for identifier in phl_games[(club, t1, t.week, t.day)]:
                        base_time = datetime.strptime(identifier[0], '%H:%M')

                        # Perform arithmetic with timedelta
                        min_time = (base_time - timedelta(minutes=120)).time()
                        max_time = (base_time + timedelta(minutes=120)).time()

                        # Convert `t.time` to datetime.time if it's a string
                        if isinstance(t.time, str):
                            t_time = datetime.strptime(t.time, '%H:%M').time()
                        else:
                            t_time = t.time  # Assume it's already a datetime.time object

                        # Now safely compare times
                        if min_time <= t_time <= max_time and identifier[1] != t.field.location:
                            phl_games[(club, t1, t.week, t.day)][identifier].append(X[key])         

                    club = get_club(t2, data['teams'])
                    for identifier in phl_games[(club, t2, t.week, t.day)]:
                        base_time = datetime.strptime(identifier[0], '%H:%M')

                        # Perform arithmetic with timedelta
                        min_time = (base_time - timedelta(minutes=120)).time()
                        max_time = (base_time + timedelta(minutes=120)).time()

                        # Convert `t.time` to datetime.time if it's a string
                        if isinstance(t.time, str):
                            t_time = datetime.strptime(t.time, '%H:%M').time()
                        else:
                            t_time = t.time  

                        # Now safely compare times
                        if min_time <= t_time <= max_time and identifier[1] != t.field.location:
                            phl_games[(club, t2, t.week, t.day)][identifier].append(X[key])  

        for date, day_slots in phl_games.items():
            for day_slot, game_vars in day_slots.items():
                    model.Add(sum(game_vars) <= 1)    

class PHLAndSecondGradeTimes(Constraint):        
    ''' PHL games should not be played at the same time. 2nds games cannot be played at the same time as PHL games within the same club. Ensure max 3 Friday night games at Broadmeadow and enforce preferred dates. 
        So we do not need to enforce this for other locations as there is only one field there. Furthermore, this means we can use day_slot not time.
    '''      
    def apply(self, model, X, data):

        if 'penalties' not in data:
            data['penalties'] = {'phl_preferences': {'weight': 10000, 'penalties': []}}
        else:
            data['penalties']['phl_preferences'] = {'weight': 10000, 'penalties': []}

        games = data['games']
        timeslots = data['timeslots']
        phl_preferences = data['phl_preferences']
        allowed_keys = {"preferred_dates"}
        invalid_keys = set(phl_preferences.keys()) - allowed_keys  # Find any keys that are not allowed

        if invalid_keys:
            raise ValueError(f"Invalid keys found: {invalid_keys}, currently do not support any keys other than {allowed_keys}")
   
        team_games = defaultdict(lambda: defaultdict(list))
        club_games = defaultdict(lambda: defaultdict(list))
        friday_games = defaultdict(list)
        preferred_dates = defaultdict(list)

        for t in timeslots:
            for (t1, t2, grade) in games:
                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)

                if grade in ['PHL', '2nd'] and key in X:
                    if grade == 'PHL':
                        # Stop PHL games across clubs from being played at the same time
                        team_games[(t.week, t.day)][(t.day_slot, t.field.location)].append(X[key])
                        
                        # Stop PHL and 2nd grade from being played at the same time, within the same club
                        club = get_club(t1, data['teams'])
                        dup_2nds = get_duplicated_graded_teams(club, '2nd', data['teams'])
                        for extra_team in dup_2nds:
                            club_games[(t.week, t.day, t.field.location)][(t.day_slot, club, extra_team)].append(X[key])

                        club = get_club(t2, data['teams'])
                        dup_2nds = get_duplicated_graded_teams(club, '2nd', data['teams'])
                        for extra_team in dup_2nds:
                            club_games[(t.week, t.day, t.field.location)][(t.day_slot, club, extra_team)].append(X[key])

                        #  Max 3 Friday night games at Newcastle International Hockey Centre
                        if t.day == 'Friday' and t.field.location == 'Newcastle International Hockey Centre':
                            friday_games['Friday'].append(X[key])
                        
                        # enforce preferred dates
                        if datetime.strptime(t.date, '%Y-%m-%d').date() in preferred_dates:
                            preferred_dates[t.date].append(X[key])

                    else:
                        
                        club = get_club(t1, data['teams'])
                        club_games[(t.week, t.day, t.field.location)][(t.day_slot, club, t1)].append(X[key])

                        club = get_club(t2, data['teams'])
                        club_games[(t.week, t.day, t.field.location)][(t.day_slot, club, t2)].append(X[key])

        for date, day_slots in team_games.items():
            for day_slot, game_vars in day_slots.items():
                if day_slot[1] == 'Newcastle International Hockey Centre': # Stop concurrent PHL games only at Broadmeadow
                    model.Add(sum(game_vars) <= 1)    
        
        for date, day_slots in club_games.items():
            for day_slot, game_vars in day_slots.items():
                if day_slot[2] == 'Newcastle International Hockey Centre': # Stop concurrent 2nd grade and PHL only at Broadmeadow 
                    model.Add(sum(game_vars) <= 1)

        for day, game_vars in friday_games.items(): # Ensure that only 3 Broadmedow games on a Friday are held
            model.Add(sum(game_vars) <= 3)

        for date, game_vars in preferred_dates.items():
            penalty_var = model.NewIntVar(0, len(game_vars), f"preferred_date_penalty_{date}")
            model.AddAbsEquality(penalty_var, sum(game_vars) - 1)
            data['penalties']['phl_preferences']['penalties'].append(penalty_var)

class NoBackToBackByes(Constraint):
    ''' Ensure there are no back to back byes'''
        
    def apply(self, model, X, data):
        games = data['games']
        timeslots = data['timeslots']
        num_rounds = data['num_rounds']

        team_games = defaultdict(lambda: defaultdict(list))
        print(f'NoBackTOBackByes skipped for 6th grade')
        for t in timeslots:
            for (t1, t2, grade) in games:
                
                if grade == '6th':
                    continue

                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                if key in X:
                    if t.round_no + 1 <= num_rounds['max']:
                        team_games[t1][(t.round_no, t.round_no + 1)].append(X[key])
                        team_games[t2][(t.round_no, t.round_no + 1)].append(X[key])

                    if t.round_no > 1:
                        team_games[t1][(t.round_no - 1, t.round_no)].append(X[key])
                        team_games[t2][(t.round_no - 1, t.round_no)].append(X[key])

        for team, time_slot in team_games.items():
            for slot, game_vars in time_slot.items():
                model.Add(sum(game_vars) >= 1)

## Soft
# class NoBackToBackByes(Constraint):
#     '''Ensure there are no back-to-back byes (soft constraint).'''
    
#     def apply(self, model, X, data):

#         if 'penalties' not in data:
#             data['penalties'] = {'NoBackToBackByes': {'weight': 100000, 'penalties': []}}
#         else:
#             data['penalties']['NoBackToBackByes'] = {'weight': 100000, 'penalties': []}

#         games = data['games']
#         timeslots = data['timeslots']
#         num_rounds = data['num_rounds']

#         team_games = defaultdict(lambda: defaultdict(list))

#         for t in timeslots:
#             for (t1, t2, grade) in games:
#                 key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
#                 if key in X:
#                     if t.week + 1 <= num_rounds['max']:
#                         team_games[t1][(t.week, t.week + 1)].append(X[key])
#                         team_games[t2][(t.week, t.week + 1)].append(X[key])

#                     if t.week > 1:
#                         team_games[t1][(t.week - 1, t.week)].append(X[key])
#                         team_games[t2][(t.week - 1, t.week)].append(X[key])

#         # Apply soft penalty for back-to-back byes
#         for team, time_slot in team_games.items():
#             for slot, game_vars in time_slot.items():
#                 penalty_var = model.NewIntVar(0, 1, f"bye_penalty_{team}_{slot}")
#                 model.Add(sum(game_vars) >= 1 - penalty_var)
#                 data['penalties']['NoBackToBackByes']['penalties'].append(penalty_var)

class FiftyFiftyHomeandAway(Constraint):
    ''' Push toward 50% of games vs each team played at home and away for each away clubs team. '''
        
    def apply(self, model, X, data):
        games = data['games']
        timeslots = data['timeslots']

        home_games = defaultdict(list)
        away_games = defaultdict(list)

        # Step 1: Collect home and away games for each team
        for t in timeslots:
            for (t1, t2, grade) in games:
                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                if ('Maitland' in t1 or 'Maitland' in t2) and key in X:
                    relevant_team = t1 if 'Maitland' in t1 else t2
                    other_team = t2 if relevant_team == t1 else t1
                    if 'Maitland' in other_team:
                        continue
                    elif t.field.location == 'Maitland Park':
                        home_games[(relevant_team, other_team)].append(X[key])
                    else:
                        away_games[(relevant_team, other_team)].append(X[key])

                if ('Gosford' in t1 or 'Gosford' in t2) and key in X:
                    relevant_team = t1 if 'Gosford' in t1 else t2
                    other_team = t2 if relevant_team == t1 else t1
                    if 'Gosford' in other_team:
                        continue
                    elif t.field.location == 'Central Coast Hockey Park':
                        home_games[(relevant_team, other_team)].append(X[key])
                    else:
                        away_games[(relevant_team, other_team)].append(X[key])
                        
        # Step 2: Apply 50/50 balancing constraint
        for team, home_vars in home_games.items():
            away_vars = away_games[team]
            
            if not home_vars or not away_vars:  # Skip teams with no games
                print(f'Team {team} has no games when calculating 50/50 home and away. Home games {home_vars}, Away games {away_vars}')  
                continue

            home_games_count = sum(home_vars)
            total_games_count = home_games_count + sum(away_vars)

            # Ensure aim_games is as close to 50% as possible

            model.Add(home_games_count * 2 >= total_games_count)  # Ensure aim_games is at least half
            model.Add(home_games_count * 2 <= total_games_count + 1)  # Allow rounding up

## Soft
"""
# class FiftyFiftyHomeandAway(Constraint):
#     ''' Push toward 50% home and away games for Maitland'''
        
    # def apply(self, model, X, data):
    #     if 'penalties' not in data:
    #         data['penalties'] = {'FiftyFiftyHomeandAway': {'weight': 10000000, 'penalties': []}}
    #     else:
    #         data['penalties']['FiftyFiftyHomeandAway'] = {'weight': 10000000, 'penalties': []}

    #     games = data['games']
    #     timeslots = data['timeslots']

    #     home_games = defaultdict(list)
    #     away_games = defaultdict(list)

    #     # Step 1: Collect home and away games for each team
    #     for t in timeslots:
    #         for (t1, t2, grade) in games:
    #             key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
    #             if ('Maitland' in t1 or 'Maitland' in t2) and key in X:
    #                 relevant_team = t1 if 'Maitland' in t1 else t2
    #                 other_team = t2 if relevant_team == t1 else t1
    #                 if 'Maitland' in other_team:
    #                     continue
    #                 elif t.field.location == 'Maitland Park':
    #                     home_games[(relevant_team, other_team)].append(X[key])
    #                 else:
    #                     away_games[(relevant_team, other_team)].append(X[key])

    #             if ('Gosford' in t1 or 'Gosford' in t2) and key in X:
    #                 relevant_team = t1 if 'Gosford' in t1 else t2
    #                 other_team = t2 if relevant_team == t1 else t1
    #                 if 'Gosford' in other_team:
    #                     continue
    #                 elif t.field.location == 'Central Coast Hockey Park':
    #                     home_games[(relevant_team, other_team)].append(X[key])
    #                 else:
    #                     away_games[(relevant_team, other_team)].append(X[key])
                        
#         # Step 2: Apply 50/50 balancing constraint
#         for team, home_vars in home_games.items():
#             away_vars = away_games[team]
            
#             if not home_vars or not away_vars:  # Skip teams with no games
#                 continue  

#             home_games_count = sum(home_vars)
#             total_games_count = home_games_count + sum(away_vars)

#             # Ensure aim_games is as close to 50% as possible
#             aim_games = model.NewIntVar(0, len(home_vars) + len(away_vars), f'aim_games_{team}')
#             model.Add(aim_games * 2 >= total_games_count)  # Ensure aim_games is at least half
#             model.Add(aim_games * 2 <= total_games_count + 1)  # Allow rounding up

#             # Define penalty variable as an absolute deviation
#             penalty_var = model.NewIntVar(0, len(home_vars) + len(away_vars), f'deviation_{team}')
#             model.AddAbsEquality(penalty_var, home_games_count - aim_games)

#             # Store penalty correctly
#             data['penalties']['FiftyFiftyHomeandAway']['penalties'].append(penalty_var)
"""

class EnsureUniqueTeamsEvery3Weeks(Constraint):
    ''' Push toward 50% of games vs each team played at home and away for each away clubs team. '''
        
    def apply(self, model, X, data):

        games = data['games']
        timeslots = data['timeslots']
        max_rounds = data['num_rounds']['max']
        game_dict = defaultdict(lambda: defaultdict(list))


        # Step 1: Collect home and away games for each team
        for t in timeslots:
            for (t1, t2, grade) in games:
                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                if key in X:
                    if t.round_no > 1 and t.round_no <= max_rounds - 1:
                        game_dict[(t1, t2)][t.round_no - 1, t.round_no, t.round_no +1].append(X[key])
                    if t.round_no > 2:
                        game_dict[(t1, t2)][t.round_no - 2, t.round_no -1 , t.round_no].append(X[key])
                    if t.round_no <= max_rounds - 2:
                        game_dict[(t1, t2)][t.round_no, t.round_no + 1, t.round_no + 2].append(X[key])

                        
        # Step 2: Apply 50/50 balancing constraint
        for team, round_group in game_dict.items():
            for round_group, game_vars in round_group.items():
                model.Add(sum(game_vars) <= 1) # For any particular pairing of teams, for any particular 3 week group, have them play once.


# SOFT CONSTRAINTS
class MaximiseClubsPerTimeslotBroadmeadow(Constraint):
    """Maximises the number of clubs per timeslot at Broadmeadow, ensuring diversity in clubs playing within the same timeslot."""

    def apply(self, model, X, data):

        if 'penalties' not in data:
            data['penalties'] = {'MaximiseClubsPerTimeslotBroadmeadow': {'weight': 5000, 'penalties': []}}
        else:
            data['penalties']['MaximiseClubsPerTimeslotBroadmeadow'] = {'weight': 5000, 'penalties': []}

        games = data['games']
        timeslots = data['timeslots']
        game_dict = defaultdict(lambda: defaultdict(list))  # { (week, day, timeslot): {club: [game_vars]} }

        for t in timeslots:
            for (t1, t2, grade) in games:
                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                
                if key in X and t.field.location == 'Newcastle International Hockey Centre':
                    club1 = get_club(t1, data['teams'])
                    club2 = get_club(t2, data['teams'])

                    game_dict[(t.week, t.day, t.day_slot)][club1].append(X[key])
                    game_dict[(t.week, t.day, t.day_slot)][club2].append(X[key])

        # Apply constraint and penalty
        for (week, day, timeslot), club_games in game_dict.items():
            # Create a binary variable for each club: 1 if club is playing, 0 otherwise
            club_presence_vars = {}
            for club, game_vars in club_games.items():
                club_var = model.NewBoolVar(f'club_{club}_week{week}_day{day}_slot{timeslot}')
                model.Add(sum(game_vars) > 0).OnlyEnforceIf(club_var)  # Club present if they play
                model.Add(sum(game_vars) == 0).OnlyEnforceIf(club_var.Not())  # Otherwise, club absent
                club_presence_vars[club] = club_var

            # Compute number of clubs playing in the timeslot
            num_clubs_var = model.NewIntVar(0, 6, f'num_clubs_week{week}_day{day}_slot{timeslot}')
            model.Add(num_clubs_var == sum(club_presence_vars.values()))

            # Define deviation penalty from ideal 6 clubs
            penalty_var = model.NewIntVar(0, 6, f'penalty_week{week}_day{day}_slot{timeslot}')
            model.AddAbsEquality(penalty_var, 6 - num_clubs_var)

            # Store penalties for minimization
            data['penalties']['MaximiseClubsPerTimeslotBroadmeadow']['penalties'].append(penalty_var)

class MinimiseClubsOnAFieldBroadmeadow(Constraint):
    """ Minimises the number of clubs playing on a field on any particular day, this way clubs get continuity of games. """

    def apply(self, model, X, data):

        if 'penalties' not in data:
            data['penalties'] = {'MinimiseClubsOnAFieldBroadmeadow': {'weight': 5000, 'penalties': []}}
        else:
            data['penalties']['MinimiseClubsOnAFieldBroadmeadow'] = {'weight': 5000, 'penalties': []}

        games = data['games']
        timeslots = data['timeslots']
        game_dict = defaultdict(lambda: defaultdict(list))  # { (week, day, timeslot): {club: [game_vars]} }

        for t in timeslots:
            for (t1, t2, grade) in games:
                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                
                if key in X and t.field.location == 'Newcastle International Hockey Centre':
                    club1 = get_club(t1, data['teams'])
                    club2 = get_club(t2, data['teams'])

                    game_dict[(t.week, t.date, t.field.name)][club1].append(X[key])
                    game_dict[(t.week, t.date, t.field.name)][club2].append(X[key])

        # Apply constraint and penalty
        for (week, day, field_name), club_games in game_dict.items():
            # Create a binary variable for each club: 1 if club is playing, 0 otherwise
            club_presence_vars = {}
            for club, game_vars in club_games.items():
                club_var = model.NewBoolVar(f'club_{club}_week{week}_day{day}_field{field_name}')
                model.Add(sum(game_vars) > 0).OnlyEnforceIf(club_var)  # Club present if they play
                model.Add(sum(game_vars) == 0).OnlyEnforceIf(club_var.Not())  # Otherwise, club absent
                club_presence_vars[club] = club_var

            # Compute number of clubs playing in the field_name
            num_clubs_var = model.NewIntVar(0, 20, f'num_clubs_week{week}_day{day}_field{field_name}')
            model.Add(num_clubs_var == sum(club_presence_vars.values()))

            # Define deviation penalty from ideal 6 clubs
            penalty_var = model.NewIntVar(0, 20, f'penalty_week{week}_day{day}_field{field_name}')
            model.AddAbsEquality(penalty_var, num_clubs_var - 2)

            # Store penalties for minimization
            data['penalties']['MinimiseClubsOnAFieldBroadmeadow']['penalties'].append(penalty_var)


class MaitlandHomeGrouping(Constraint):
    '''Encourage all of Maitland's games to be at home or away as a group each week.'''

    def apply(self, model, X, data):

        if 'penalties' not in data:
            data['penalties'] = {'MaitlandHomeGrouping': {'weight': 100000, 'penalties': []}}
        else:
            data['penalties']['MaitlandHomeGrouping'] = {'weight': 100000, 'penalties': []}

        # Track Maitland's total games per week and home games per week
        maitland_games_per_week = {}
        maitland_home_games_per_week = {}

        for t in data['timeslots']:
            for (t1, t2, grade) in data['games']:
                if "Maitland" in t1 or "Maitland" in t2:
                    key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                    if key in X:
                        week = t.week
                        if week not in maitland_games_per_week:
                            maitland_games_per_week[week] = []
                            maitland_home_games_per_week[week] = []

                        game_var = X[key]  # Decision variable for whether this game is scheduled
                        maitland_games_per_week[week].append(game_var)

                        if t.field.location == "Maitland Park":
                            maitland_home_games_per_week[week].append(game_var)

        for week in maitland_games_per_week:
            if not maitland_games_per_week[week]:  
                continue  # Skip weeks with no games

            max_games_per_week = len(data['timeslots'])  # Safe upper bound

            total_games_var = model.NewIntVar(0, max_games_per_week, f'total_games_week_{week}')
            home_games_var = model.NewIntVar(0, max_games_per_week, f'home_games_week_{week}')
            
            # Compute total and home games
            model.Add(total_games_var == sum(maitland_games_per_week[week]))
            model.Add(home_games_var == sum(maitland_home_games_per_week[week]))

            # Approximate half of total_games_var safely (avoid division errors)
            half_games_floor = model.NewIntVar(0, max_games_per_week // 2, f'half_games_floor_week_{week}')
            half_games_ceil = model.NewIntVar(0, max_games_per_week // 2 + 1, f'half_games_ceil_week_{week}')

            model.Add(half_games_floor * 2 <= total_games_var)  # Lower bound of half
            model.Add(half_games_ceil * 2 >= total_games_var)  # Upper bound of half

            # Compute absolute deviation from 50%
            delta_var = model.NewIntVar(0, max_games_per_week // 2, f'delta_week_{week}')
            model.AddAbsEquality(delta_var, home_games_var - half_games_floor)

            # Define V-shaped penalty: maximum at 50%, minimum at 0% and 100%
            imbalance_penalty = model.NewIntVar(0, max_games_per_week, f'imbalance_penalty_week_{week}')
            model.Add(imbalance_penalty == half_games_ceil - delta_var)

            # Store penalty
            data['penalties']['MaitlandHomeGrouping']['penalties'].append(imbalance_penalty)

class AwayAtMaitlandGrouping(Constraint):
    '''Encourage all of a club's games against Maitland to be played at Maitland when they are the away team.'''

    def apply(self, model, X, data):

        if 'penalties' not in data:
            data['penalties'] = {'AwayAtMaitlandGrouping': {'weight': 8000, 'penalties': []}}
        else:
            data['penalties']['AwayAtMaitlandGrouping'] = {'weight': 8000, 'penalties': []}

        # Track away clubs playing Maitland in each week
        away_clubs_per_week = defaultdict(lambda: defaultdict(list))  # {week: {club: [game_var1, game_var2, ...]}}

        for t in data['timeslots']:
            for (t1, t2, grade) in data['games']:
                if "Maitland Park" in t.field.location:
                    key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)

                    if key in X:  # Only consider scheduled games
                        away_club = get_club(t1, data['teams']) if "Maitland" in t2 else get_club(t2, data['teams'])  
                        away_clubs_per_week[t.week][away_club].append(X[key])  # Store (club, game decision var)

        # Apply constraint to encourage a single away club per week at Maitland
        for week, club_games in away_clubs_per_week.items():
            club_scheduled_vars = {}  # Track whether each club has at least one game scheduled that week

            for club, game_vars in club_games.items():
                # Create a binary variable that is 1 if the club has at least one scheduled game
                club_scheduled_var = model.NewBoolVar(f'club_{club}_week_{week}_scheduled')
                model.Add(sum(game_vars) > 0).OnlyEnforceIf(club_scheduled_var)  # Activate if at least one game is scheduled
                model.Add(sum(game_vars) == 0).OnlyEnforceIf(club_scheduled_var.Not())  # Otherwise, set to 0
                club_scheduled_vars[club] = club_scheduled_var

            # Define a solver variable to track the number of distinct clubs playing at Maitland
            num_clubs_var = model.NewIntVar(0, len(club_scheduled_vars), f'num_away_clubs_week{week}')
            model.Add(num_clubs_var == sum(club_scheduled_vars.values()))  # Count distinct clubs dynamically


            # Apply penalty if more than one club is playing at Maitland in the same week
            penalty_var = model.NewIntVar(0, len(club_scheduled_vars), f'penalty_week{week}_away_club_mismatch')
            model.Add(penalty_var == num_clubs_var - 1)  # Linearly increase penalty as num_clubs_var grows

            # Store penalties for minimization
            data['penalties']['AwayAtMaitlandGrouping']['penalties'].append(penalty_var)

class MaitlandGradeOrder(Constraint):
    '''Encourage games in Maitland to be scheduled in grade order (7th to PHL).'''

    def apply(self, model, X, data):
        if 'penalties' not in data:
            data['penalties'] = {'MaitlandGradeOrder': {'weight': 50000, 'penalties': []}}
        else:
            data['penalties']['MaitlandGradeOrder'] = {'weight': 50000, 'penalties': []}

        expected_order = ["7th", "6th", "5th", "4th", "3rd", "2nd", "PHL"]
        grade_to_rank = {grade: rank for rank, grade in enumerate(expected_order)}

        # Step 1: Collect all Maitland games per day
        maitland_games = defaultdict(lambda: defaultdict(list))  # {week: {day: [(day_slot, grade, X[key])]}}

        for t in data['timeslots']:
            for (t1, t2, grade) in data['games']:
                if t.field.location == "Maitland Park":
                    key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                    if key in X:
                        maitland_games[t.week][t.day].append((t.day_slot, grade, X[key]))

        # Step 2: Apply a penalty for disorder using weighted sum
        for week, days in maitland_games.items():
            for day, games in days.items():
                games.sort(key=lambda game: game[0])  # Sort by day_slot

                # Convert grades to their indices in the expected order
                grade_ranks = [grade_to_rank[grade] for _, grade, _ in games if grade in grade_to_rank]

                # Calculate disorder penalty based on sum of weighted ranks
                penalty_var = model.NewIntVar(0, sum(range(len(grade_ranks))), f'grade_order_penalty_week{week}_day{day}')
                model.Add(penalty_var == sum(grade_ranks[i] * games[i][2] for i in range(len(grade_ranks))))

                data['penalties']['MaitlandGradeOrder']['penalties'].append(penalty_var)


class MinimizeLateGames(Constraint):
    '''Encourage games at all fields to be scheduled back-to-back and earlier in the day.'''

    def apply(self, model, X, data):
        if 'penalties' not in data:
            data['penalties'] = {'MinimizeLateGames': {'weight': 10000, 'penalties': []}}
        else:
            data['penalties']['MinimizeLateGames'] = {'weight': 10000, 'penalties': []}

        games_by_field = defaultdict(lambda: defaultdict(list))  # {week: {field: [(day_slot, game_var)]}}

        # Step 1: Collect games per field per week
        for t in data['timeslots']:
            for (t1, t2, grade) in data['games']:
                key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                if key in X:
                    games_by_field[(t.week, t.day)][t.field.name].append((t.day_slot, X[key]))

        # Step 2: Apply constraints per field per week
        for week, fields in games_by_field.items():
            for field, games in fields.items():
                # Sort games by `day_slot`
                games.sort(key=lambda g: g[0])  
                game_vars = [var for _, var in games]
                day_slots = [slot for slot, _ in games]

                # Define a variable for the number of scheduled games that day
                num_games_var = model.NewIntVar(0, len(game_vars), f'num_games_week{week}_field{field}')
                model.Add(num_games_var == sum(game_vars))  # Count number of scheduled games

                max_games = len(game_vars)  # Upper bound

                # Define a variable for the actual sum of day slots
                actual_sum_var = model.NewIntVar(0, max_games, f'actual_sum_week{week}_field{field}')
                model.Add(actual_sum_var == sum(slot * var for slot, var in games))

                # Define expected sum based on the formula S(n) = n(n+1)/2
                temp_var = model.NewIntVar(0, max_games * (max_games + 1), f'temp_mult_week{week}_field{field}')
                model.AddMultiplicationEquality(temp_var, num_games_var, num_games_var + 1)

                expected_sum_var = model.NewIntVar(0, (max_games * (max_games + 1)) // 2, f'expected_sum_week{week}_field{field}')
                model.AddDivisionEquality(expected_sum_var, temp_var, 2)  # expected_sum = (n(n+1)) / 2

                # Define penalty variable for excess slots
                penalty_var = model.NewIntVar(0, sum(day_slots), f'late_game_penalty_week{week}_field{field}')
                model.Add(penalty_var == actual_sum_var - expected_sum_var)  # Penalize deviation

                # Store penalties
                data['penalties']['MinimizeLateGames']['penalties'].append(penalty_var)

class MaximiseSundayGames(Constraint):
    '''Encourage games to be scheduled on Sundays by applying a negative penalty.'''

    def apply(self, model, X, data):
        if 'penalties' not in data:
            data['penalties'] = {'RewardSundayGames': {'weight': -8000, 'penalties': []}}
        else:
            data['penalties']['RewardSundayGames'] = {'weight': -8000, 'penalties': []}

        sunday_games_by_field = defaultdict(lambda: defaultdict(list))  # {week: {field: [game_var]}}

        # Step 1: Collect games that are scheduled on Sundays
        for t in data['timeslots']:
            if t.day == "Sunday":  # Only consider Sunday games
                for (t1, t2, grade) in data['games']:
                    key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
                    if key in X:
                        sunday_games_by_field[(t.week, t.day)][t.field.name].append(X[key])

        # Step 2: Reward Sunday games per field per week
        for week, fields in sunday_games_by_field.items():
            for field, game_vars in fields.items():
                # Define a variable for the total number of games scheduled on Sundays
                num_sunday_games_var = model.NewIntVar(0, len(game_vars), f'sunday_games_week{week}_field{field}')
                model.Add(num_sunday_games_var == sum(game_vars))  # Count number of Sunday games

                # Define a "penalty" variable, but since weight is negative, it's actually a reward
                reward_var = model.NewIntVar(0, len(game_vars), f'sunday_reward_week{week}_field{field}')
                model.Add(reward_var == num_sunday_games_var)  # More Sunday games, higher reward

                # Store reward (negative penalty) to encourage scheduling games on Sundays
                data['penalties']['RewardSundayGames']['penalties'].append(reward_var)

# Soft or User set constraints
class PreferredTimesConstraint(Constraint):
    """Ensure teams play at preferred times and avoid no-play times."""
    
    def apply(self, model, X, data):

        if 'penalties' not in data:
            data['penalties'] = {'PreferredTimesConstraint': {'weight': 100000, 'penalties': []}}
        else:
            data['penalties']['PreferredTimesConstraint'] = {'weight': 100000, 'penalties': []}

        teams = data['teams']
        clubs = data['clubs']
        noplay = data['preference_no_play']
        
        allowed_keys = ['team_name', 'team2', 'grade', 'day', 'day_slot', 'time', 'week', 'date', 'field_name', 'field_location']
        
        allowed_keys2 = ['team1', 'team_name', 'grade', 'day', 'day_slot', 'time', 'week', 'date', 'field_name', 'field_location']

        # Enforce no-play times with penalties
        for club_name, restrictions in noplay.items():
            club_teams = get_teams_from_club(club_name, teams)
            if club_name.lower() not in [c.name.lower() for c in clubs]:
                raise ValueError(f'Invalid team name {club_name} in PreferredTimeConstraint')

            
            for index, constraint in enumerate(restrictions):
                if not all(key in allowed_keys for key in constraint.keys()):
                    raise ValueError(f"Invalid key in noplay constraint for {club_name}: {constraint.keys()}")
                
                # Find matching games in X
                for game_key in X:
                    game_dict = dict(zip(allowed_keys, game_key))
                    if all(game_dict.get(k) == v for k, v in constraint.items()) and (game_key[0] in club_teams or game_key[1] in club_teams):
                        penalty_var = model.NewIntVar(0, 1, f"penalty_{club_name}_{index}")
                        model.Add(X[game_key] <= 1 - penalty_var)
                        data['penalties']['PreferredTimesConstraint']['penalties'].append(penalty_var)

                    game_dict = dict(zip(allowed_keys2, game_key))
                    if all(game_dict.get(k) == v for k, v in constraint.items()) and (game_key[0] in club_teams or game_key[1] in club_teams):
                        penalty_var = model.NewIntVar(0, 1, f"penalty_{club_name}_{index}")
                        model.Add(X[game_key] <= 1 - penalty_var)
                        data['penalties']['PreferredTimesConstraint']['penalties'].append(penalty_var)
                        
class TeamConflictConstraint(Constraint):
    ''' Ensure that when clubs specify two teams cannot play at the same time, they do not.'''

    def apply(self, model, X, data):
        conflicts = data['team_conflicts']
        fields = data['fields']

        for team_pairing in conflicts:
            team1 = team_pairing[0]
            team2 = team_pairing[1]
            for t in data['timeslots']:
                game_vars = []

                for (t1, t2, grade) in data['games']:
                    if t1 in [team1, team2] or t2 in [team1, team2]:
                        for field in fields:
                            key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, field.name, field.location)
                            if key in X:
                                game_vars.append(X[key])
                model.Add(sum(game_vars) <= 1)
                     
class ClubDayConstraint(Constraint):
    """ This consstraint deals with ensuring club days occur correctly. """
    
    def apply(self, model, X, data):

        if 'penalties' not in data:
            data['penalties'] = {'ClubDayConstraint': {'weight': 1000000, 'penalties': []}}
        else:
            data['penalties']['ClubDayConstraint'] = {'weight': 1000000, 'penalties': []}

        club_days = data['club_days']
        teams = data['teams']
        clubs = data['clubs']
        
        allowed_keys = ['team1', 'team2', 'grade', 'day', 'day_slot', 'time', 'week', 'date', 'field_name', 'field_location']

        for club_name in club_days:   
            if club_name.lower() not in [c.name.lower() for c in clubs]:
                raise ValueError(f'Invalid team name {club_name} in ClubDay Dictionary')   

            desired_date = club_days[club_name]
            club = get_club_from_clubname(club_name, data['clubs'])
            club_teams = get_teams_from_club(club_name, teams)
            home_field = club.home_field
            
            # Locate all games for the club on the desired date
            club_games = [key for key in X if key[allowed_keys.index('date')] == desired_date.date().strftime('%Y-%m-%d')  
                          and (key[allowed_keys.index('team1')] in club_teams 
                               or key[allowed_keys.index('team2')] in club_teams)]
            
            if not club_games:
                raise ValueError(f"No games found for club {club_name} on {desired_date.date()}")
            
            # Track which fields are used dynamically
            field_usage_vars = {}
            for game_key in club_games:
                field_name = game_key[allowed_keys.index('field_name')]
                if field_name not in field_usage_vars:
                    field_usage_vars[field_name] = model.NewBoolVar(f'field_used_{club_name}_{field_name}')
                
                # Ensure this field is "used" if any game on it is scheduled
                model.Add(X[game_key] <= field_usage_vars[field_name])

            # Count the number of distinct fields used dynamically
            num_fields_used = model.NewIntVar(0, len(field_usage_vars), f'num_fields_used_{club_name}')
            model.Add(num_fields_used == sum(field_usage_vars.values()))

            # Penalize if more than one field is used
            penalty_var1 = model.NewIntVar(0, len(field_usage_vars), f'club_field_penalty_{club_name}')
            model.Add(penalty_var1 == num_fields_used - 1)
            data['penalties']['ClubDayConstraint']['penalties'].append(penalty_var1)

            
            # Track which teams are scheduled dynamically
            team_scheduled_vars = {}
            for team in club_teams:
                team_scheduled_vars[team] = model.NewBoolVar(f'team_scheduled_{club_name}_{team}')

            # Ensure a team's variable is "1" if they are scheduled in any game
            for game_key in club_games:
                team1, team2 = game_key[allowed_keys.index('team1')], game_key[allowed_keys.index('team2')]
                if team1 in team_scheduled_vars:
                    model.Add(X[game_key] <= team_scheduled_vars[team1])
                if team2 in team_scheduled_vars:
                    model.Add(X[game_key] <= team_scheduled_vars[team2])

            # Count the number of scheduled club teams dynamically
            num_scheduled_teams = model.NewIntVar(0, len(club_teams), f'num_scheduled_teams_{club_name}')
            model.Add(num_scheduled_teams == sum(team_scheduled_vars.values()))

            # Penalize if not all club teams are scheduled
            penalty_var2 = model.NewIntVar(0, len(club_teams), f'club_teams_penalty_{club_name}')
            model.Add(penalty_var2 == len(club_teams) - num_scheduled_teams)
            data['penalties']['ClubDayConstraint']['penalties'].append(penalty_var2)




                
"""

# class MaitlandGradeOrder(Constraint):
#     '''Encourage games in Maitland to be scheduled in grade order (6th to PHL).'''

#     def apply(self, model, X, data):
#         if 'penalties' not in data:
#             data['penalties'] = {'MaitlandGradeOrder': {'weight': 50000, 'penalties': []}}
#         else:
#             data['penalties']['MaitlandGradeOrder'] = {'weight': 50000, 'penalties': []}

#         expected_order = ["7th", "6th", "5th", "4th", "3rd", "2nd", "PHL"]  

#         # Step 1: Collect all Maitland games per day
#         maitland_games = defaultdict(lambda: defaultdict(list))  # {week: {day: [(day_slot, grade, X[key])]}}

#         for t in data['timeslots']:
#             for (t1, t2, grade) in data['games']:
#                 if t.field.location == "Maitland":
#                     key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
#                     if key in X:
#                         maitland_games[t.week][t.day].append((t.day_slot, grade, X[key]))

#         # Step 2: Apply penalties for incorrect ordering
#         for week, days in maitland_games.items():
#             for day, games in days.items():
#                 # Sort by day_slot (earliest games first)
#                 games.sort(key=lambda game: game[0])  # Sorting by day_slot ensures correct order

#                 # Extract grades and corresponding decision variables
#                 scheduled_grades = [grade for _, grade, _ in games]
#                 decision_vars = [game_var for _, _, game_var in games]

#                 # Convert grades to their indices in the expected order
#                 indices = [expected_order.index(grade) for grade in scheduled_grades if grade in expected_order]

#                 # Create a penalty variable to track out-of-order games
#                 penalty_var = model.NewIntVar(0, len(indices)**2, f'grade_order_penalty_week{week}_day{day}')

#                 # Penalize based on how out of order the grades are
#                 penalty_terms = []
#                 for i in range(len(indices) - 1):
#                     for j in range(i + 1, len(indices)):
#                         # If a lower-grade index appears after a higher-grade index, penalize
#                         out_of_order = model.NewBoolVar(f'grade_order_violation_{week}_{day}_{i}_{j}')
#                         model.Add(indices[i] > indices[j]).OnlyEnforceIf(out_of_order)  
#                         penalty_terms.append(out_of_order)

#                 model.Add(penalty_var == sum(penalty_terms))  # Sum of all misorderings

#                 data['penalties']['MaitlandGradeOrder']['penalties'].append(penalty_var)
# """



"""
# class PHLAndSecondGradeAdjacency(Constraint):
#     '''Ensure that PHL and 2nds do not play in adjacent day_slots at different locations.''' 
    
#     def apply(self, model, X, data):
#         teams = data['teams']
#         interested_grades = {'PHL', '2nd'}

#         if 'penalties' not in data:
#             data['penalties'] = {'PHLAndSecondGradeAdjacency': {'weight': 1000, 'penalties': []}}
#         else:
#             data['penalties']['PHLAndSecondGradeAdjacency'] = {'weight': 1000, 'penalties': []}

#         for t in data['timeslots']:
#             for (t1, t2, grade) in data['games']:
#                 if grade not in interested_grades:
#                     continue
                
#                 key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
#                 if key not in X:
#                     continue
                
#                 club1 = get_club(t1, teams)
#                 club2 = get_club(t2, teams)
#                 game_var = X[key]
                
#                 # Check adjacent time slots for PHL/2nd grade games
#                 for adj_t in data['timeslots']:
#                     if adj_t.day == t.day and abs(adj_t.day_slot - t.day_slot) == 1:
#                         for (adj_t1, adj_t2, adj_grade) in data['games']:
#                             if adj_grade not in interested_grades or adj_grade == grade:
#                                 continue  # Ensure it's the other grade (PHL or 2nd)

#                             adj_key = (adj_t1, adj_t2, adj_grade, adj_t.day, adj_t.day_slot, adj_t.time, adj_t.week, adj_t.date, adj_t.field.name, adj_t.field.location)
#                             if adj_key not in X:
#                                 continue
                            
#                             adj_club1 = get_club(adj_t1, teams)
#                             adj_club2 = get_club(adj_t2, teams)
#                             adj_game_var = X[adj_key]

#                             # If fields are different, and clubs match, penalize adjacency
#                             if t.field.location != adj_t.field.location and (
#                                 adj_club1 in [club1, club2] or adj_club2 in [club1, club2]
#                             ):
#                                 penalty_var = model.NewBoolVar(f'penalty_{key}_{adj_key}')
                                
#                                 # Penalize only if both games are scheduled
#                                 model.Add(game_var + adj_game_var == 2).OnlyEnforceIf(penalty_var)
                                
#                                 # Store penalty in data
#                                 data['penalties']['PHLAndSecondGradeAdjacency']['penalties'].append(penalty_var)
# """

"""
# class MaitlandHomeGameBalance(Constraint):
#     '''Encourage each Maitland team to plays 50% of its games at home.'''
    
#     def apply(self, model, X, data):
#         if 'penalties' not in data:
#             data['penalties'] = {'MaitlandHomeGameBalance': {'weight': 100000, 'penalties': []}}
#         else:
#             data['penalties']['MaitlandHomeGameBalance'] = {'weight': 100000, 'penalties': []}

#         maitland_total_games = defaultdict(list)  # Store game decision variables for total games
#         maitland_home_games = defaultdict(list)   # Store game decision variables for home games

#         # Step 1: Collect decision variables for games involving Maitland teams
#         for t in data['timeslots']:
#             for (t1, t2, grade) in data['games']:
#                 if "Maitland" in t1 or "Maitland" in t2:
#                     key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)

#                     if key in X:  # Only consider scheduled games
#                         if "Maitland" in t1:
#                             maitland_total_games[t1].append(X[key])  # Store decision variable
#                             if t.field.location == "Maitland":
#                                 maitland_home_games[t1].append(X[key])  # Store decision variable if home
#                         if "Maitland" in t2:
#                             maitland_total_games[t2].append(X[key])  # Store decision variable
#                             if t.field.location == "Maitland":
#                                 maitland_home_games[t2].append(X[key])  # Store decision variable if home

#         # Step 2: Apply constraints for each Maitland team
#         for team in maitland_total_games:
#             total_games_var = model.NewIntVar(0, len(maitland_total_games[team]), f"{team}_total_games")
#             home_games_var = model.NewIntVar(0, len(maitland_total_games[team]), f"{team}_home_games")
#             penalty_var_max = model.NewIntVar(0, len(maitland_total_games[team]), f"{team}_home_penalty_max")
#             penalty_var_min = model.NewIntVar(0, len(maitland_total_games[team]), f"{team}_home_penalty_min")

#             max_expr_var = model.NewIntVar(0, len(maitland_total_games[team]), f"{team}_max_expr")
#             min_expr_var = model.NewIntVar(0, len(maitland_total_games[team]), f"{team}_min_expr")


#             # Sum up actual scheduled games (total and home)
#             model.Add(total_games_var == sum(maitland_total_games[team]))
#             model.Add(home_games_var == sum(maitland_home_games[team]))

#             # Calculate the 50% threshold
#             home_threshold = len(maitland_total_games[team]) // 2  # 50% of total games (rounded down)

#             # Compute max(home_threshold - home_games_var, 0)
#             model.AddMaxEquality(max_expr_var, [home_threshold - home_games_var, 0])
#             model.Add(penalty_var_max == max_expr_var)

#             # Enforce maximum home games threshold (penalize if above threshold)
#             model.AddMinEquality(min_expr_var, [home_games_var - home_threshold, 0])
#             model.Add(penalty_var_min == min_expr_var)

#             # Add penalties to data
#             data['penalties']['MaitlandHomeGameBalance']['penalties'].extend([penalty_var_min, penalty_var_max])
# """

"""
# # class AwayTeamsPlayHomeEverySecondWeek(Constraint):
# #     '''Every second week Maitland plays at home.'''

# #     def apply(self, model, X, data):
# #         games = data['games']
# #         timeslots = data['timeslots']
# #         num_rounds = data['num_rounds']

# #         team_home_games = defaultdict(dict)  # Track home games per team per week

# #         if 'penalties' not in data:
# #             data['penalties'] = {'AwayTeamsPlayHomeEverySecondWeek': {'weight': 1000, 'penalties': []}}
# #         else:
# #             data['penalties']['AwayTeamsPlayHomeEverySecondWeek'] = {'weight': 1000, 'penalties': []}

# #         # Step 1: Track home games per team per week
# #         for t in timeslots:
# #             for (t1, t2, grade) in games:
# #                 key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
# #                 club1 = get_club_object(t1, data['teams'])
# #                 club2 = get_club_object(t2, data['teams'])
                
# #                 # Get home field info
# #                 home_field = (club1.home_field, club2.home_field)

# #                 # Ignore Newcastle International Hockey Centre teams
# #                 if any(field == "Maitland" for field in home_field) and t.field.location == 'Maitland':
                    
# #                     # Determine the relevant team that is playing at home
# #                     if t.field.location == club1.home_field:
# #                         relevant_team = t1
# #                     elif t.field.location == club2.home_field:
# #                         relevant_team = t2
# #                     else:
# #                         continue  # Neither team is playing at home

# #                     if key in X:
# #                         team_home_games[relevant_team][t.week] = X[key]  # Store game variable

# #         # Step 2: Enforce "home one week, away the next" rule
# #         for team, weekly_games in team_home_games.items():
# #             for week in range(1, num_rounds):  # Check all weeks
# #                 if week in weekly_games and (week + 1) in weekly_games:
# #                     violation = model.NewIntVar(0, 1, f"violation_{team}_week_{week}")
# #                     model.Add(weekly_games[week] + weekly_games[week + 1] <= 1).OnlyEnforceIf(violation)  # Penalize back-to-back home games
                    
# #                     data['penalties']['AwayTeamsPlayHomeEverySecondWeek']['penalties'].append(violation)  # Store penalty variable
# """

"""
# class ClubBackToBackGames(Constraint):
#     '''Encourage a club's games to be scheduled back-to-back on the same day.'''
#     def apply(self, model, X, data):
#         if 'penalties' not in data:
#             data['penalties'] = {'ClubBackToBackGames': {'weight': 8, 'penalties': []}}
        
#         for t in data['timeslots']:
#             for (t1, t2, grade) in data['games']:
#                 club = get_club(t1, data['teams'])
#                 next_slot = t.day_slot + 1
#                 for adj_t in data['timeslots']:
#                     if adj_t.day_slot == next_slot:
#                         adj_key = (t1, t2, grade, adj_t.day, adj_t.day_slot, adj_t.time, adj_t.week, adj_t.date, adj_t.field.name, adj_t.field.location)
#                         if adj_key in X:
#                             penalty_var = model.NewBoolVar(f'penalty_{t1}_{t2}_back_to_back')
#                             model.Add(penalty_var >= X[adj_key] + X[key] - 1)
#                             data['penalties']['ClubBackToBackGames']['penalties'].append(penalty_var)


# """

"""
# class AwayTeamsPlayHomeEverySecondWeek(Constraint):
#     '''Every Second week Maitland plays at home'''
        
#     def apply(self, model, X, data):
#         games = data['games']
#         timeslots = data['timeslots']
#         num_rounds = data['num_rounds']

#         team_games = defaultdict(lambda: defaultdict(list))

#         if 'penalties' not in data:
#             data['penalties'] = {'AwayTeamsPlayHomeEverySecondWeek': {'weight': 1000, 'penalties': []}}
#         else:
#             data['penalties']['AwayTeamsPlayHomeEverySecondWeek'] = {'weight': 1000, 'penalties': []}

#         for t in timeslots:
#             for (t1, t2, grade) in games:
#                 key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
#                 club1 = get_club_object(t1, data['teams'])
#                 club2 = get_club_object(t2, data['teams'])
#                 home_field = (club1.home_field, club2.home_field)

#                 if any(field == 'Maitland' for field in home_field) and t.field.location == 'Maitland' and key in X:

#                     relevant_team = t1 if t.field.location == club1.home_field else t2

#                     if t.week + 1 <= num_rounds:
#                         team_games[relevant_team][(t.week, t.week + 1)].append(X[key])
#                     if t.week > 1:
#                         team_games[relevant_team][(t.week - 1, t.week)].append(X[key])

#         for team, time_slot in team_games.items():
#             for time_slot, game_vars in time_slot.items():
#                 violation = model.NewIntVar(0, 1, f"violation_{team}_{time_slot}")
#                 model.Add(sum(game_vars) == 1).OnlyEnforceIf(violation.Not())  # Constraint holds when no violation
#                 data['penalties']['AwayTeamsPlayHomeEverySecondWeek']['penalties'].append(violation)  # Store violation variable

# """

"""
# class MaitlandHomeGrouping(Constraint):
#     '''Encourage all of Maitland's games to be at home if at least one is scheduled at home that weekend.'''

#     def apply(self, model, X, data):

#         if 'penalties' not in data:
#             data['penalties'] = {'MaitlandHomeGrouping': {'weight': 1000, 'penalties': []}}
#         else:
#             data['penalties']['MaitlandHomeGrouping'] = {'weight': 1000, 'penalties': []}

#         maitland_home_weeks = set()  # Track weeks where Maitland actually plays at home

#         # Step 1: Identify weeks where Maitland has at least one scheduled home game
#         for t in data['timeslots']:
#             for (t1, t2, grade) in data['games']:
#                 if ("Maitland" in t1 or "Maitland" in t2) and t.field.location == "Maitland":
#                     key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
#                     if key in X:
#                         is_scheduled = model.NewBoolVar(f'scheduled_{t.week}_{t1}_{t2}')
#                         model.Add(X[key] == 1).OnlyEnforceIf(is_scheduled)  # Ensure this game is actually scheduled
#                         model.Add(X[key] == 0).OnlyEnforceIf(is_scheduled.Not())

#                         # We use a helper variable to track if any game is scheduled at Maitland this week
#                         maitland_home_weeks.add(t.week)  

#         # Step 2: Penalize Maitland games that are NOT at home during those weeks
#         for t in data['timeslots']:
#             for (t1, t2, grade) in data['games']:
#                 if ("Maitland" in t1 or "Maitland" in t2) and t.week in maitland_home_weeks and t.field.location != "Maitland":
#                     key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
#                     if key in X:
#                         penalty_var = model.NewBoolVar(f'penalty_week{t.week}_maitland_away')
#                         model.Add(X[key] == 1).OnlyEnforceIf(penalty_var)  # Apply penalty only if game is scheduled
#                         data['penalties']['MaitlandHomeGrouping']['penalties'].append(penalty_var)
# """

"""
# class MaitlandHomeGrouping(Constraint):
#     '''Encourage all of Maitland's games to be at home or away as a group each week.'''

#     def apply(self, model, X, data):

#         if 'penalties' not in data:
#             data['penalties'] = {'MaitlandHomeGrouping': {'weight': 10000, 'penalties': []}}
#         else:
#             data['penalties']['MaitlandHomeGrouping'] = {'weight': 10000, 'penalties': []}

#         # Track Maitland's total games per week and home games per week
#         maitland_games_per_week = {}
#         maitland_home_games_per_week = {}

#         for t in data['timeslots']:
#             for (t1, t2, grade) in data['games']:
#                 if "Maitland" in t1 or "Maitland" in t2:
#                     key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
#                     if key in X:
#                         week = t.week
#                         if week not in maitland_games_per_week:
#                             maitland_games_per_week[week] = []
#                             maitland_home_games_per_week[week] = []

#                         game_var = X[key]  # Decision variable for whether this game is scheduled
#                         maitland_games_per_week[week].append(game_var)

#                         if t.field.location == "Maitland":
#                             maitland_home_games_per_week[week].append(game_var)

#         # Apply proportional constraints
#         for week in maitland_games_per_week:
#             # Define total games as a solver variable (tracks all scheduled Maitland games)
#             total_games_var = model.NewIntVar(0, len(maitland_games_per_week[week]), f'total_games_week_{week}')
#             model.Add(total_games_var == sum(maitland_games_per_week[week]))  # Dynamically updates as games are scheduled

#             # Define home games as a solver variable (tracks scheduled home games)
#             home_games_var = model.NewIntVar(0, len(maitland_home_games_per_week[week]), f'home_games_week_{week}')
#             model.Add(home_games_var == sum(maitland_home_games_per_week[week]) )  # Dynamically updates as games are scheduled
            
#             # Define a helper variable to represent total_games_var // 2
#             half_total_games_var = model.NewIntVar(0, len(maitland_games_per_week[week]) // 2, f'half_total_games_week_{week}')
#             model.Add(half_total_games_var * 2 <= total_games_var)  # Enforces integer division behavior

#             imbalance_var = model.NewIntVar(0, total_games_var // 2, f'imbalance_var_week_{week}')
#             model.AddAbsEquality(imbalance_var, home_games_var - total_games_var // 2)

#             quadratic_penalty = model.NewIntVar(0, (total_games_var // 2) ** 2, f'quadratic_penalty_week_{week}')
#             model.AddMultiplicationEquality(quadratic_penalty, [imbalance_var, imbalance_var])

#             if len(maitland_games_per_week[week]) > 0:
#                 # Soft constraints to discourage mixed home/away scheduling
#                 partial_home_penalty = model.NewBoolVar(f'partial_home_week_{week}')
#                 model.Add(home_games_var > half_total_games_var).OnlyEnforceIf(partial_home_penalty)
#                 model.Add(home_games_var < total_games_var).OnlyEnforceIf(partial_home_penalty)

#                 partial_away_penalty = model.NewBoolVar(f'partial_away_week_{week}')
#                 model.Add(home_games_var <= half_total_games_var).OnlyEnforceIf(partial_away_penalty)
#                 model.Add(home_games_var > 0).OnlyEnforceIf(partial_away_penalty)

#                 # Store penalties for minimization
#                 data['penalties']['MaitlandHomeGrouping']['penalties'].append(partial_home_penalty)
#                 data['penalties']['MaitlandHomeGrouping']['penalties'].append(partial_away_penalty)

# """

"""
# class MinimizeLateGames(Constraint):
#     '''Encourage games at all fields to be scheduled back-to-back and earlier in the day.'''

#     def apply(self, model, X, data):
#         if 'penalties' not in data:
#             data['penalties'] = {'MinimizeLateGames': {'weight': 10000, 'penalties': []}}
#         else:
#             data['penalties']['MinimizeLateGames'] = {'weight': 10000, 'penalties': []}

#         # Step 1: Collect all games per field per day, with their times
#         games_by_field = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))  # {week: {field: {day: [(time, X[key])]}}}

#         for t in data['timeslots']:
#             for (t1, t2, grade) in data['games']:
#                 key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
#                 if key in X:
#                     games_by_field[t.week][t.field.name][t.day].append((t.time, X[key]))

#         # Step 2: Identify gaps and late games
#         for week, fields in games_by_field.items():
#             for field, days in fields.items():
#                 for day, games in days.items():
#                     # Sort games by time (earliest first)
#                     games.sort(key=lambda game: game[0]) 
                    
#                     # Convert string times to time objects safely
#                     try:
#                         times = [tm.fromisoformat(time) for time, _ in games]
#                     except ValueError:
#                         try:
#                             times = [datetime.strptime(time, "%H:%M").time() for time, _ in games]  
#                         except ValueError:
#                             raise ValueError(f"Invalid time format for field {field} on day {day} in week {week}")

#                     game_vars = [var for _, var in games]  # Extract game decision variables

#                     # Penalize large gaps between games
#                     for i in range(len(times) - 1):
#                         gap = (datetime.combine(datetime.today(), times[i + 1]) - 
#                                datetime.combine(datetime.today(), times[i])).seconds // 60  # Gap in minutes

#                         if gap > 90:  # Assume 90 min is the max acceptable gap
#                             penalty_var = model.NewBoolVar(f'penalty_week{week}_field{field}_day{day}_gap_{i}')
                            
#                             # Apply penalty only if both games are scheduled
#                             model.Add(game_vars[i] + game_vars[i + 1] == 2).OnlyEnforceIf(penalty_var)
                            
#                             data['penalties']['MinimizeLateGames']['penalties'].append(penalty_var)

#                     # Penalize last games if they are too late
#                     if times and times[-1] > tm(15, 0):  # Now correctly checking after 3 PM
#                         penalty_var = model.NewBoolVar(f'penalty_week{week}_field{field}_day{day}_late_game')

#                         # Apply penalty only if the late game is scheduled
#                         model.Add(game_vars[-1] == 1).OnlyEnforceIf(penalty_var)

#                         data['penalties']['MinimizeLateGames']['penalties'].append(penalty_var)
# """

"""
# # class AwayTeamsPlayHomeEverySecondWeek(Constraint):
# #     ''' Ensures that away teams play at home every second week'''
        
# #     def apply(self, model, X, data):
# #         games = data['games']
# #         timeslots = data['timeslots']
# #         num_rounds = data['num_rounds']

# #         home_games = defaultdict(lambda: defaultdict(list))
# #         away_games = defaultdict(lambda: defaultdict(list))


# #         for t in timeslots:
# #             for (t1, t2, grade) in games:
# #                 key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
# #                 if ('Maitland' in t1 or 'Maitland' in t2) and key in X:

# #                     relevant_team = t1 if 'Maitland' in t1 else t2
# #                     if t.field.location == 'Maitland':
# #                         if t.week + 1 <= num_rounds:
# #                             home_games[relevant_team][(t.week, t.week + 1)].append(X[key])

# #                         if t.week > 1:
# #                             home_games[relevant_team][(t.week - 1, t.week)].append(X[key])
# #                     else:
# #                         if t.week + 1 <= num_rounds:
# #                             away_games[relevant_team][(t.week, t.week + 1)].append(X[key])

# #                         if t.week > 1:
# #                             away_games[relevant_team][(t.week - 1, t.week)].append(X[key])

# #         for team, time_slot in home_games.items():
# #             for time_slot, game_vars in time_slot.items():
# #                 model.Add(sum(game_vars) <= 1)

# #         for team, time_slot in away_games.items():
# #             for time_slot, game_vars in time_slot.items():
# #                 model.Add(sum(game_vars) <= 1)
# # """

"""
# class FiftyFiftyHomeandAway(Constraint):
#     ''' Push toward 50% home and away games for Maitland'''
        
#     def apply(self, model, X, data):
#         if 'penalties' not in data:
#             data['penalties'] = {'FiftyFiftyHomeandAway': {'weight': 10000, 'penalties': []}}
#         else:
#             data['penalties']['FiftyFiftyHomeandAway'] = {'weight': 10000, 'penalties': []}

#         games = data['games']
#         timeslots = data['timeslots']

#         home_games = defaultdict(list)
#         away_games = defaultdict(list)

#         # Step 1: Collect home and away games for each team
#         for t in timeslots:
#             for (t1, t2, grade) in games:
#                 key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
#                 if ('Maitland' in t1 or 'Maitland' in t2) and key in X:
#                     relevant_team = t1 if 'Maitland' in t1 else t2
#                     if t.field.location == 'Maitland':
#                         home_games[relevant_team].append(X[key])
#                     else:
#                         away_games[relevant_team].append(X[key])

#         # Step 2: Apply 50/50 balancing constraint
#         for team, home_vars in home_games.items():
#             away_vars = away_games[team]
            
#             if not home_vars or not away_vars:  # Skip teams with no games
#                 continue  

#             home_games_count = sum(home_vars)
#             total_games_count = home_games_count + sum(away_vars)
#             aim_games = model.NewIntVar(0, len(home_vars) + len(away_vars), f'aim_games_{team}')

#             # Ensure aim_games is as close to 50% as possible
#             model.Add(aim_games * 2 >= total_games_count)  # Ensure aim_games is at least half
#             model.Add(aim_games * 2 <= total_games_count + 1)  # Allow rounding up

#             # Penalize deviation from ideal 50/50 split
#             penalty_var = model.NewIntVar(0, len(home_vars) + len(away_vars), f'penalty_{team}')
#             model.AddAbsEquality(penalty_var, home_games_count - aim_games)

#             # Store penalty correctly
#             data['penalties']['FiftyFiftyHomeandAway']['penalties'].append(penalty_var)
# """

"""
# class MaitlandGradeOrder(Constraint):
#     '''Encourage games in Maitland to be scheduled in grade order (6th to PHL).'''

#     def apply(self, model, X, data):

#         if 'penalties' not in data:
#             data['penalties'] = {'MaitlandGradeOrder': {'weight': 1000, 'penalties': []}}
#         else:
#             data['penalties']['MaitlandGradeOrder'] = {'weight': 1000, 'penalties': []}

#         expected_order = ["6th", "5th", "4th", "3rd", "2nd", "PHL"]  

#         # Step 1: Collect all Maitland games per day
#         maitland_games = defaultdict(lambda: defaultdict(list))  # {week: {day: [(day_slot, grade, X[key])]}}

#         for t in data['timeslots']:
#             for (t1, t2, grade) in data['games']:
#                 if t.field.location == "Maitland":
#                     key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)
#                     if key in X:
#                         maitland_games[t.week][t.day].append((t.day_slot, grade, X[key]))

#         # Step 2: Check and penalize incorrect grade order per day
#         for week, days in maitland_games.items():
#             for day, games in days.items():
#                 # Sort by day_slot (earliest games first)
#                 games.sort(key=lambda game: game[0])  # Sorting by day_slot ensures correct order

#                 # Extract grades and corresponding decision variables
#                 scheduled_grades = [grade for _, grade, _ in games]
#                 decision_vars = [game_var for _, _, game_var in games]

#                 # Convert grades to their indices in the expected order
#                 indices = [expected_order.index(grade) for grade in scheduled_grades if grade in expected_order]

#                 # If grades are **not in the expected order**, penalize
#                 if indices != sorted(indices):
#                     penalty_var = model.NewBoolVar(f'penalty_week{week}_day{day}_maitland_grade_order')

#                     # If games are scheduled in the wrong order, the penalty should be applied
#                     model.Add(sum(decision_vars) > 0).OnlyEnforceIf(penalty_var)

#                     data['penalties']['MaitlandGradeOrder']['penalties'].append(penalty_var)

"""


'\n# class MaitlandGradeOrder(Constraint):\n#     \'\'\'Encourage games in Maitland to be scheduled in grade order (6th to PHL).\'\'\'\n\n#     def apply(self, model, X, data):\n\n#         if \'penalties\' not in data:\n#             data[\'penalties\'] = {\'MaitlandGradeOrder\': {\'weight\': 1000, \'penalties\': []}}\n#         else:\n#             data[\'penalties\'][\'MaitlandGradeOrder\'] = {\'weight\': 1000, \'penalties\': []}\n\n#         expected_order = ["6th", "5th", "4th", "3rd", "2nd", "PHL"]  \n\n#         # Step 1: Collect all Maitland games per day\n#         maitland_games = defaultdict(lambda: defaultdict(list))  # {week: {day: [(day_slot, grade, X[key])]}}\n\n#         for t in data[\'timeslots\']:\n#             for (t1, t2, grade) in data[\'games\']:\n#                 if t.field.location == "Maitland":\n#                     key = (t1, t2, grade, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location)\n#                     if key in 

In [23]:
def generate_timeslots(start_date, end_date, day_time_map, fields, field_unavailabilities):
    """Generate weekly timeslots between two dates, considering field unavailability."""
    timeslots = []
    current_date = start_date
    week_number = 1
    round_no = 0
    day_slot = 1
    c_day = None
    round_indic = True
    draw_start = False

    # Check if all fields in field_unavailabilities exist in fields
    field_name_check = list(field_unavailabilities.keys())
    known_fields = [field.location for field in fields]
    for field_name in field_name_check:
        if field_name not in known_fields:
            raise ValueError(f"Field {field_name} in field_unavailabilities does not exist in fields!") 
    # Check that fields are correct in day time map
    for field_name in day_time_map.keys():
        if field_name not in known_fields:
            raise ValueError(f"Field {field_name} in day_time_map does not exist in fields!")

    while current_date <= end_date:
        day_name = current_date.strftime('%A')
        if day_name in [key for field in day_time_map for key in day_time_map[field].keys()]:
            draw_start = True
            if c_day != day_name:
                day_slot = 1
                c_day = day_name

            # Check if the whole weekend is out for Broadmeadow to set the rounds
            if any(current_date.date() in [(w - timedelta(days=1)).date(), w.date(), (w + timedelta(days=1)).date()] 
                for w in field_unavailabilities.get('Newcastle International Hockey Centre', {}).get('weekends', [])):
                pass
            elif round_indic:
                round_no += 1
                round_indic = False


            for field in fields:
                field_name = field.location

                # Check if the whole weekend is unavailable (compare dates only)
                if any(current_date.date() in [(w - timedelta(days=1)).date(), w.date(), (w + timedelta(days=1)).date()] 
                    for w in field_unavailabilities.get(field_name, {}).get('weekends', [])):
                    continue

                day_slot = 1
                
                for t in day_time_map[field_name][day_name]:
            
                    # Check if the whole day is unavailable (compare dates only)
                    if current_date.date() in [d.date() for d in field_unavailabilities.get(field_name, {}).get('whole_days', [])]:
                        continue
                    
                    # Check if a partial day is unavailable (compare both date and time)
                    if any(current_date.date() == pd.date() and t == pd.time() 
                        for pd in field_unavailabilities.get(field_name, {}).get('part_days', [])):
                        continue



                    timeslots.append({
                        'date': current_date.strftime('%Y-%m-%d'),
                        'day': day_name,
                        'time': t.strftime('%H:%M'),
                        'week': week_number,
                        'day_slot': day_slot,
                        'field': field,
                        'round_no': round_no
                    })
                    day_slot += 1

        if current_date.strftime('%A') == 'Monday' and draw_start:
            week_number += 1
            day_slot = 1
            round_indic = True

        current_date += timedelta(days=1)

    print(f'Total rounds found are {round_no}.')

    cur_week = 0
    count = 0
    for slot in timeslots:
        if slot['week'] != cur_week:
            count += 1
            cur_week = slot['week']

    print(f'Number of weekends with games on is {count}')
    return timeslots

def generate_X(folder_path, model, data):
    """
    Optimized version of game filtering to improve performance while tracking unavailable games.
    """
    # Create games dictionary efficiently
    games = {
        (t1.name, t2.name, t1.grade): (t1, t2, t1.grade)
        for i, t1 in enumerate(data['teams'])
        for t2 in data['teams'][i + 1:] if t1.grade == t2.grade
    }
    data['games'] = games
    print(f"Generated {len(games)} games.")  
    
    # Generate X variables
    timeslots = data['timeslots']
    teams = data['teams']
    phl_game_times = data['phl_game_times']
    day_time_map = data['day_time_map']


    X = {
        (t1.name, t2.name, grade_name, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location):
        model.NewBoolVar(f'X_{t1}_{t2}_{t.day}_{t.time}_{t.week}_{t.field.name}_{t.field.location}')
        for (t1_name, t2_name, grade_name), (t1, t2, grade) in games.items()
        for t in timeslots
        if t.field.location in {t1.club.home_field, t2.club.home_field}
        if grade_name != 'PHL' 
        if ('Maitland' not in t1.name and 'Maitland' not in t2.name)
        if t.day in day_time_map[t.field.location] and datetime.strptime(t.time, "%H:%M").time() in day_time_map[t.field.location][t.day]
    }

    print(f'Decision variables {len(X)}')

    maitland_X = {
        (t1.name, t2.name, grade_name, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location):
        model.NewBoolVar(f'X_{t1}_{t2}_{t.day}_{t.time}_{t.week}_{t.field.name}_{t.field.location}')
        for (t1_name, t2_name, grade_name), (t1, t2, grade) in games.items()
        for t in timeslots
        if t.field.location in {t1.club.home_field, t2.club.home_field} # Means that clubs that have multiple teams in the same grade will only play grudge matches at home
        if grade_name != 'PHL' 
        if ('Maitland' in t1.name or 'Maitland' in t2.name)
        if t.day in day_time_map[t.field.location] and datetime.strptime(t.time, "%H:%M").time() in day_time_map[t.field.location][t.day]

# Conditions
        if t.day != 'Saturday' # Rule out all Maitland games on a saturday
    }

    print(f'Maitland decision variables {len(maitland_X)}')

    phl_X = {
        (t1.name, t2.name, grade_name, t.day, t.day_slot, t.time, t.week, t.date, t.round_no, t.field.name, t.field.location):
        model.NewBoolVar(f'X_{t1}_{t2}_{t.day}_{t.time}_{t.week}_{t.field.name}_{t.field.location}')
        for (t1_name, t2_name, grade_name), (t1, t2, grade) in games.items()
        for t in timeslots
        if grade_name == 'PHL'
        if t.day in phl_game_times[t.field.location] and datetime.strptime(t.time, "%H:%M").time() in phl_game_times[t.field.location][t.day] # Allow phl to play at only special times
        if t.field.location in {t1.club.home_field, t2.club.home_field}
# Conditions
        if t.field.name != 'SF' # Rule out south field
        if not (t.field.location == 'Maitland Park' and t.day == 'Friday') # No friday night games at maitland
        if not (t.field.location == 'Central Coast Hockey Park' and t.day == 'Friday' and datetime.strptime(t.date, '%Y-%m-%d') < datetime(2025, 5, 1))  # Do not schedule Friday games at Wyong before May 1st
        if not datetime.strptime(t.date, '%Y-%m-%d') in [datetime(2025, 5, 9), datetime(2025, 5, 23), datetime(2025, 6, 6), datetime(2025, 6, 27), datetime(2025, 7, 4)] # These are dates the Juniors convenor said no to
        if (t.day != 'Friday' or (t.day == 'Friday' and 
            t1.name in ['Gosford PHL', 'Maitland PHL', 'Tigers PHL', 'Wests PHL', 'Souths PHL'] and 
            t2.name in ['Gosford PHL', 'Maitland PHL', 'Tigers PHL', 'Wests PHL', 'Souths PHL'] and
            t.field.location == 'Central Coast Hockey Park')) # Set it so that only the above teams play at Wyong on a Friday
        if (t.day != 'Friday' or (t.day == 'Friday' and 
            t1.name in ['Maitland PHL', 'Tigers PHL', 'Wests PHL', 'Souths PHL'] and 
            t2.name in ['Maitland PHL', 'Tigers PHL', 'Wests PHL', 'Souths PHL'] and
            t.field.location == 'Newcastle International Hockey Centre')) # Only clubs with Juniors versions are to play on a friday night at Broadmeadow
    }

    print(f'PHL decision variables {len(phl_X)}')
    
    X.update(maitland_X)
    X.update(phl_X)
    
    filtered_games = set(X.keys())  # Using a set for faster lookups
    unavailable_games = set()
    conflicting_matchups = []

    club_names = [c.name.lower() for c in data['clubs']]

    for file in os.listdir(folder_path):
        if not file.endswith("_noplay.xlsx"):
            continue
        
        club_name = file.split("_noplay.xlsx")[0]
        club_teams = get_teams_from_club(club_name, teams)

        file_path = os.path.join(folder_path, file)

        if club_name.lower() not in club_names:
            raise ValueError(f"Club {club_name} in {file} does not exist in clubs!")
        
        if '~$' in club_name: # Filter out any temporary files
            continue
        
        # Read only required columns and convert dates in bulk
        club_noplay = pd.read_excel(file_path, sheet_name="club_noplay", usecols=["whole_weekend", "whole_day", "timeslot"])
        teams_noplay = pd.read_excel(file_path, sheet_name="teams_noplay", usecols=["team", "whole_weekend", "whole_day", "timeslot"])
        team_conflicts = pd.read_excel(file_path, sheet_name="team_conflicts", usecols=["team1", "team2"])
        
        for df in [club_noplay, teams_noplay]:
            df.fillna(np.nan, inplace=True)
            df["whole_weekend"] = pd.to_datetime(df["whole_weekend"], format="%d/%m/%Y", errors="coerce")
            df["whole_day"] = pd.to_datetime(df["whole_day"], format="%d/%m/%Y", errors="coerce")
            df["timeslot"] = pd.to_datetime(df["timeslot"], format="%d/%m/%Y %H:%M", errors="coerce")
                
        # Remove club-wide restricted times efficiently
        for _, row in club_noplay.iterrows():
            if pd.notna(row["whole_weekend"]):
                week_num = row["whole_weekend"].isocalendar()[1]
                to_remove = {k for k in filtered_games if datetime.strptime(k[7], "%Y-%m-%d").isocalendar()[1] == week_num and (k[0] in club_teams or k[1] in club_teams)}
                unavailable_games.update(to_remove)
                filtered_games -= to_remove
            if pd.notna(row["whole_day"]):
                day_date = row["whole_day"].date()
                to_remove = {k for k in filtered_games if datetime.strptime(k[7], "%Y-%m-%d").date() == day_date and (k[0] in club_teams or k[1] in club_teams)}
                unavailable_games.update(to_remove)
                filtered_games -= to_remove
            if pd.notna(row["timeslot"]):
                date_time = row["timeslot"]
                to_remove = {k for k in filtered_games if datetime.strptime(k[7], "%Y-%m-%d").date() == date_time.date() and k[5] == date_time.strftime("%H:%M") and (k[0] in club_teams or k[1] in club_teams)}
                unavailable_games.update(to_remove)
                filtered_games -= to_remove
        
        # Remove team-specific restrictions efficiently
        valid_teams = {k[0].lower() for k in filtered_games} | {k[1].lower() for k in filtered_games}
        
        for _, row in teams_noplay.iterrows():
            team = row["team"]
            if team.lower() not in valid_teams:
                raise ValueError(f"Team {team} in {file} does not exist in games!")
            
            if pd.notna(row["whole_weekend"]):
                week_num = row["whole_weekend"].isocalendar()[1]
                to_remove = {k for k in filtered_games if team in k[:2] and datetime.strptime(k[7], "%Y-%m-%d").isocalendar()[1] == week_num}
                unavailable_games.update(to_remove)
                filtered_games -= to_remove
            if pd.notna(row["whole_day"]):
                day_date = row["whole_day"].date()
                to_remove = {k for k in filtered_games if team in k[:2] and datetime.strptime(k[7], "%Y-%m-%d").date() == day_date}
                unavailable_games.update(to_remove)
                filtered_games -= to_remove
            if pd.notna(row["timeslot"]):
                date_time = row["timeslot"]
                to_remove = {k for k in filtered_games if team in k[:2] and datetime.strptime(k[7], "%Y-%m-%d").date() == date_time.date() and k[5] == date_time.strftime("%H:%M")}
                unavailable_games.update(to_remove)
                filtered_games -= to_remove
        
        # Record conflicting team matchups
        conflicting_matchups.extend(list(team_conflicts.itertuples(index=False, name=None)))
        
        # Validate that all teams in conflicting_matchups exist in valid_teams
        for team1, team2 in conflicting_matchups:
            if team1.lower() not in valid_teams or team2.lower() not in valid_teams:
                raise ValueError(f"Conflicting teams {team1} and {team2} are not in the valid teams list!")

    
    return {k: X[k] for k in filtered_games}, conflicting_matchups, unavailable_games

def create_schedule(X, data, constraints, model, conflicts):
    print(f"Generated {len(X)} decision variables.")  
    
    # Apply constraints dynamically
    
    for constraint in constraints:
        constraint.apply(model, X, data)
    
    penalties_dict = data.get('penalties', {})

    print(f'Number of penalties per constraint: {[(name, len(info.get("penalties", []))) for name, info in penalties_dict.items()]}')

    total_penalty = sum(
        info['weight'] * sum(info['penalties'])  # Multiply weight by sum of violations
        for info in penalties_dict.values() if 'penalties' in info
    )
    # Maximize scheduled games while penalizing adjacent games at different locations
    model.Maximize(
        sum(X.values()) - total_penalty  # Subtract the weighted penalties
    )
    print(f"Total constraints in model: {len(model.Proto().constraints)}")
    
    solver = cp_model.CpSolver()

    solver.parameters.max_time_in_seconds = 900 #14400 #43200 #82800 # this last one was close, maybe double it and be ok

    
    status = solver.Solve(model)

    print(f"Solver status: {solver.StatusName()}") 

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        solution = {key: solver.Value(var) for key, var in X.items()}

        for constraint_name, info in data.get('penalties', {}).items():
            if 'penalties' in info:
                unresolved_penalties = [p for p in info['penalties'] if solver.Value(p) > 0]
                data['penalties'][constraint_name]['unresolved'] = unresolved_penalties

        return solution, data
    else:
        print("No feasible schedule found.")
        return {}, data

day_time_map = {'Newcastle International Hockey Centre':
                    { 
                        'Saturday':[tm(12, 30), tm(14, 00), tm(15, 30), tm(17, 00)],
                     'Sunday':[tm(8, 30), tm(10, 0), tm(11,30), tm(13, 00), tm(14, 30), tm(16, 00), tm(17, 30), tm(19, 00)]
                     },
                'Maitland Park':
                    {
                     'Sunday':[tm(9, 00), tm(10, 30), tm(12, 00), tm(13, 30), tm(15, 00), tm(16, 30)]
                     }
}

assert len(day_time_map['Maitland Park']['Sunday']) >= get_club_from_clubname('Maitland', CLUBS).num_teams, "Not enough timeslots for Maitland teams!"

phl_game_times = {'Newcastle International Hockey Centre':
                    {'Friday':[tm(19, 00)],
                    'Sunday':[tm(11,30), tm(13, 00), tm(14, 30), tm(16, 00)]
                    },    
                    'Central Coast Hockey Park':
                        {'Friday':[tm(20, 00)],
                         'Sunday': [tm(15, 00)]
                         },  
                    'Maitland Park':
                    {
                     'Sunday':[ tm(12, 00), tm(13, 30), tm(15, 00), tm(16, 30)]
                     }
                    } 

start = datetime(2025, 3, 21) # Set start date a few days before weekend
end = datetime(2025, 9, 2)  # Set end date a days after weekend

weekend_count = sum(1 for i in range((end - start).days + 1)
                    if (start + timedelta(days=i)).weekday() in {5, 6}) / 2 # 5 = Saturday, 6 = Sunday

print(f"Number of weekends: {weekend_count}")

field_unavailabilities = {'Maitland Park': 
                            {'weekends':[datetime(2025, 4, 19), datetime(2025, 4, 12), datetime(2025, 5, 10), datetime(2025, 5, 24), datetime(2025, 6, 28), datetime(2025,5,3), datetime(2025,6,7) ],  # Enter the saturday of the weekend to be safe and definitely catch Fridays as being out too #
                            'whole_days':[datetime(2025, 4, 25),],
                            'part_days':[],
                            },
                        'Newcastle International Hockey Centre': 
                            {'weekends':[datetime(2025, 4, 19), datetime(2025,5,3), datetime(2025,6,7)],
                            'whole_days':[datetime(2025, 4, 25), datetime(2025, 5, 31)],
                            'part_days':[datetime(2025, 6,1,8,30), datetime(2025, 6, 1, 10,0), datetime(2025,6,1,11,30), ],
                            },
                        'Central Coast Hockey Park': 
                            {'weekends':[datetime(2025, 4, 19), datetime(2025,5,3), datetime(2025,6,7), datetime(2025, 4, 5)],
                            'whole_days':[datetime(2025, 4, 25), ],
                            'part_days':[],
                            },
                        }
"""
For field unavailabilities, enter the SATURDAY to rule out the whole weekend.
"""
club_days = {'Crusaders': datetime(2025, 5, 24),
             'Wests': datetime(2025, 7, 13),
             'University': datetime(2025, 7, 27),
             'Tigers': datetime(2025, 7, 5)
             }

"""
On club days, pick one field and schedule all games there, prioritise morning and grade order
"""

preference_no_play = {'Maitland':[{'date':datetime(2025, 7, 20), 'field_location':'Newcastle International Hockey Centre'},
                                  {'date':datetime(2025, 8, 24), 'field_location':'Newcastle International Hockey Centre' },
                                    ],
                    'Norths':[{'team_name': 'Norths PHL', 'date':datetime(2025, 3, 23), 'time':tm(11,30)},
                              {'team_name': 'Norths PHL', 'date':datetime(2025, 3, 23), 'time':tm(13,00)},
                              {'team_name': 'Norths PHL', 'date':datetime(2025, 3, 23), 'time':tm(14,30)},
                              {'team_name': 'Norths PHL', 'date':datetime(2025, 3, 23), 'time':tm(16,00)}
                              ],
                      
                      }
  
phl_preferences = {'preferred_dates' :[datetime(2025, 5, 16), datetime(2025, 6, 20)]}
"""
On preference_no_play, use same keys as either Field or Timeslot attributes, special exemption for field_location and field_name, but if specifying a team, must enter the key team_name
"""
merged_dict = defaultdict(lambda: defaultdict(list))

num_rounds = {'2nd': 20, '3rd': 20, '4th': 20, '5th': 20, '6th': 20, 'PHL': 20, 'max': 21}

# Iterate over both dictionaries
for d in (phl_game_times, day_time_map):
    for field, days in d.items():
        for key, times in days.items():
            if field in merged_dict and key in merged_dict[field]:
                merged_dict[field][key].extend(times)
            else:
                merged_dict[field][key] = list(times)

for field in merged_dict:
    for key in merged_dict[field]:
        merged_dict[field][key] = list(dict.fromkeys(merged_dict[field][key]))
        merged_dict[field][key].sort()
    
timeslots = generate_timeslots(start, end, merged_dict, FIELDS, field_unavailabilities)

TIMESLOTS = [Timeslot(date=t['date'], day=t['day'], time=t['time'], week=t['week'], day_slot=t['day_slot'], field=t['field'], round_no=t['round_no']) for t in timeslots]
max_day_slot_per_field = {field.location: max(t.day_slot for t in TIMESLOTS if t.field.location == field.location) for field in FIELDS}
UNAVAILABILITY_PATH = r'C:\Users\c3205\Documents\Code\Jupyter Notebooks\draw\data\2025\noplay'


Number of weekends: 24.0
Total rounds found are 21.
Number of weekends with games on is 21


In [14]:
# Instantiate data
data = {'teams': TEAMS, 'grades': GRADES, 'fields': FIELDS, 'timeslots': TIMESLOTS, 'clubs': CLUBS, 'num_rounds': num_rounds, 'day_time_map': day_time_map, 'phl_game_times': phl_game_times, 'phl_preferences': phl_preferences, 'max_day_slot_per_field': max_day_slot_per_field, 'field_unavailabilities': field_unavailabilities, 'club_days': club_days}
model = cp_model.CpModel()
X_final, conflicts, unavailable_games = generate_X(UNAVAILABILITY_PATH, model, data)
data['unavailable_games'] = unavailable_games
data['preference_no_play'] = preference_no_play
data['team_conflicts'] = conflicts



Generated 179 games.
Decision variables 93345
Maitland decision variables 22089
PHL decision variables 2764


In [15]:
# Maitland 2nds worked FiftyFiftyHomeandAway(), MinimizeLateGames(), NoDoubleBookingTeamsConstraint(), NoDoubleBookingFieldsConstraint(),  EnsureEqualGamesAndBalanceMatchUps(), introduced extra TeamConflictConstraint(), PreferredTimesConstraint(), NoBackToBackByes() , ClubDayConstraint()
constraints = [NoDoubleBookingTeamsConstraint(), NoDoubleBookingFieldsConstraint(),  EnsureEqualGamesAndBalanceMatchUps(), PHLAndSecondGradeAdjacency(), PHLAndSecondGradeTimes(), NoBackToBackByes(), FiftyFiftyHomeandAway(), EnsureUniqueTeamsEvery3Weeks(), MinimiseClubsOnAFieldBroadmeadow(), MaximiseClubsPerTimeslotBroadmeadow()]#ClubDayConstraint(), MinimizeLateGames(), TeamConflictConstraint(), PreferredTimesConstraint(), MaitlandHomeGrouping(), AwayAtMaitlandGrouping(),  MaitlandGradeOrder()]

X_, data = create_schedule(X=X_final, data=data, constraints=constraints, model=model, conflicts=conflicts)


Generated 117388 decision variables.
Aim total games for grade 4th is 16.
Aim total games for grade 5th is 18.
Aim total games for grade 6th is 14.
Aim total games for grade 3rd is 18.
Aim total games for grade 2nd is 16.
Aim total games for grade PHL is 20.
NoBackTOBackByes skipped for 6th grade
Number of penalties per constraint: [('phl_preferences', 0), ('MinimiseClubsOnAFieldBroadmeadow', 123), ('MaximiseClubsPerTimeslotBroadmeadow', 245)]
Total constraints in model: 14652
Solver status: FEASIBLE


In [16]:
X = X_

In [ ]:
def get_field_by_name(name, FIELDS):
    for field in FIELDS:
        if field.name == name:
            return field  
    raise ValueError(f"Field {name} not found in field list.") 

def get_grade_by_name(name, GRADE):
    for grade in GRADE:
        if grade.name == name:
            return grade  
    raise ValueError(f"Grade {name} not found in grade list.")

def convert_X_to_roster(X: Dict, data: Dict) -> Roster:
    weekly_games: Dict[int, List[Game]] = {}
    all_teams: Set[str] = {team.name for team in data['teams']}  # Extract all team names

    for (team1, team2, grade, day, day_slot, time, week, date, round_no, field_name, location), var in X.items():
        if var > 0:
            field_class = get_field_by_name(field_name, data['fields'])
            grade_class = get_grade_by_name(grade, data['grades'])
            game = Game(
                team1=team1,
                team2=team2,
                timeslot=Timeslot(date=date, day=day, time=time, week=week, field=PlayingField(location=location, name=field_name), day_slot=day_slot, round_no=round_no),
                field=field_class,
                grade=grade_class
            )
            weekly_games.setdefault((week, round_no), []).append(game)

    weekly_draws = []
    for (week, round_no), games in sorted(weekly_games.items()):
        teams_played = {game.team1 for game in games} | {game.team2 for game in games}
        bye_teams = list(all_teams - teams_played)  # Teams not playing this week
        weekly_draws.append(WeeklyDraw(week=week, games=games, bye_teams=bye_teams, round_no=round_no))

    return Roster(weeks=weekly_draws)

def export_roster_to_excel(roster: Roster, filename="schedule.xlsx"):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        for weekly_draw in roster.weeks:
            week = weekly_draw.week
            data = [
                [game.team1, game.team2, game.field, game.timeslot.day, game.timeslot.time]
                for game in weekly_draw.games
            ]

            df = pd.DataFrame(data, columns=["Team 1", "Team 2", "Field Name", "Day", "Time"])
            df.to_excel(writer, sheet_name=f"Week {week}", index=False)

def check_back_to_back_byes(roster: Roster) -> List[str]:
    """
    Ensures that no team has back-to-back byes in the season schedule.
    Returns a list of teams that have consecutive byes.
    """
    previous_week = []
    current_week = []
    violations = {}
    curr_round = 1
    for weekly_draw in roster.weeks:
        if weekly_draw.bye_teams:
            if weekly_draw.round_no == 1:
                previous_week = weekly_draw.bye_teams
                continue
            else:
                if weekly_draw.round_no != curr_round:
                    current_week = weekly_draw.bye_teams
                    if any(team in previous_week for team in current_week):
                        violations[weekly_draw.week] = [team for team in current_week if team in previous_week]
                    previous_week = current_week
                    curr_round = weekly_draw.round_no

    return violations

def check_maitland_complex_limitations(roster: Roster, data: Dict) -> List[str]:
    """
    Check to see that when maitland plays at home, they all play at home
    Check to see that when maitland at home, the grades are played in order
    Check that when maitland plays at home, that all of one club comes up for the day
    """
    teams = data['teams']
    clubs = data['clubs']
    violations = {}
    weekly_proportions = []
    away_game_clubs_list = []
    maitland_team_order_list = []

    for weekly_draw in roster.weeks:

        away_game_clubs = defaultdict(list) # Maitland plays only one club at home
        maitland_team_order = defaultdict(lambda: defaultdict(list)) # Maitland plays in order of grade
        team_order_helper = defaultdict(lambda: defaultdict(list))
        maitland_location = defaultdict(list) # Maitland plays at home
        
        for game in weekly_draw.games:
            if game.field.location == 'Maitland Park':
                maitland_location['Home'].append(game)

                club = get_club(game.team1, teams)
                if club != 'Maitland':
                    away_game_clubs[club].append(game)
                else:
                    club = get_club(game.team2, teams)
                    away_game_clubs[club].append(game)
                    
                if 'Maitland' in game.team1:
                    maitland_team_order[game.timeslot.day][game.timeslot.day_slot].append(game.grade)
                    team_order_helper[game.timeslot.day][game.timeslot.day_slot].append(game.grade.name)
                elif 'Maitland' in  game.team2:
                    maitland_team_order[game.timeslot.day][game.timeslot.day_slot].append(game.grade)
                    team_order_helper[game.timeslot.day][game.timeslot.day_slot].append(game.grade.name)

            elif 'Maitland' in game.team1 or 'Maitland' in  game.team2:
                maitland_location['Away'].append(game)
                
                
        # Process home proportion requirement
        weekly_proportion = 0
        home_var = 0
        away_var = 0
        for location, games in maitland_location.items():
            if location == 'Home':
                home_var = len(games)
            elif location == 'Away':
                away_var = len(games)
        weekly_proportion = home_var / (home_var + away_var) if (home_var + away_var) > 0 else np.nan

        weekly_proportions.append((weekly_draw.week, weekly_proportion))

        # Process away game club requirement
        count = []
        for club, games in away_game_clubs.items():
            count.append((club, len(games)))
        
        away_game_clubs_list.append((weekly_draw.week, count))
        
        # Process grade progression requirement
        team_order = []
        for day in maitland_team_order.keys():
            for i, day_slot in enumerate(sorted(maitland_team_order[day].keys())):
                if i == 0:
                    prev_grade = maitland_team_order[day][day_slot]
                else:
                    if prev_grade > maitland_team_order[day][day_slot]:
                        team_order.append((day_slot, prev_grade[0].name, maitland_team_order[day][day_slot][0].name))
                    prev_grade = maitland_team_order[day][day_slot]
        maitland_team_order_list.append((weekly_draw.week, team_order))
  

    violations['Weekly Proportions'] = weekly_proportions   
    violations['Away Game Clubs'] = away_game_clubs_list
    violations['Maitland Team Order'] = maitland_team_order_list
    return violations

def check_home_proportion(roster: Roster, data: dict):
    """
    Check that Maitland plays 50% of games at home
    """
   
    maitland_teams = defaultdict(lambda: defaultdict(list))

    violations = {}
    for weekly_draw in roster.weeks:
        for game in weekly_draw.games:
            
            if 'Maitland' in game.team1:
                if 'Maitland Park' in game.field.location:
                    maitland_teams[game.team1]['Home'].append(game)
                else:
                    maitland_teams[game.team1]['Away'].append(game)

            elif 'Maitland' in game.team2:
                if 'Maitland Park' in game.field.location:
                    maitland_teams[game.team2]['Home'].append(game)
                else:
                    maitland_teams[game.team2]['Away'].append(game)

            if 'Gosford' in game.team1:
                if 'Central Coast Hockey Park' in game.field.location:
                    maitland_teams[game.team1]['Home'].append(game)
                else:
                    maitland_teams[game.team1]['Away'].append(game)

            elif 'Gosford' in game.team2:
                if 'Central Coast Hockey Park' in game.field.location:
                    maitland_teams[game.team2]['Home'].append(game)
                else:
                    maitland_teams[game.team2]['Away'].append(game)

    for key in maitland_teams.keys():
        home_games = len(maitland_teams[key]['Home'])
        total_games = home_games + len(maitland_teams[key]['Away'])
        
        violations[key] = home_games / total_games 
        
    return violations

def check_no_draw_gaps(roster: Roster, data: Dict) -> List[str]:
    """
    Check that there are no gaps in the draw (games should be consecutive for each field)
    """
    teams = data['teams']
    clubs = data['clubs']
    violations = {}
    weekly_draws = []

    # Group games by week and field
    for weekly_draw in roster.weeks:
        weekly_games = defaultdict(lambda: defaultdict(list))
        for game in weekly_draw.games:
            field_name = game.timeslot.field.name
            weekly_games[field_name][game.timeslot.day].append(game)

        # For each field, check the day_slot sequence
        for field_name, days in weekly_games.items():
            for day, games  in days.items():
                # Sort games by day_slot
                sorted_games = sorted(games, key=lambda g: g.timeslot.day_slot)
                day_slots = [game.timeslot.day_slot for game in sorted_games]
            
            # Check if day_slots are continuous and start from 1
            expected_day_slots = list(range(1, len(day_slots) + 1))
            if day_slots != expected_day_slots:
                if weekly_draw.week not in violations:
                    violations[weekly_draw.week] = []
                violations[weekly_draw.week].append((field_name, day_slots))

    return violations

def check_roster_for_requested_unavailability(roster, unavailable_games, field_unavailabilities):
    violations = []
    
    for week in roster.weeks:
        for game in week.games:
            key = (game.team1, game.team2, game.grade.name, game.timeslot.day, game.timeslot.day_slot, game.timeslot.time, game.timeslot.week, game.timeslot.date, game.field.name, game.field.location)
            if key in unavailable_games:
                violations.append(f"Game {game} was scheduled but should have been unavailable.")
            
            if game.field.location in field_unavailabilities:
                field_info = field_unavailabilities[game.field.location]
                game_date = datetime.strptime(game.timeslot.date, "%Y-%m-%d").date()  # Convert to date object
                game_iso_week = game_date.isocalendar()[1]  # Get ISO week number
                # Check if the game is in an unavailable weekend
                for weekend in field_info.get('weekends', []):
                    if game_iso_week == weekend.isocalendar()[1] :
                        violations.append(f"Game {game} was scheduled on an unavailable weekend.")

                # Check for whole-day unavailability
                if game_date in [d.date() for d in field_info.get('whole_days', [])]:
                    violations.append(f"Game {game} was scheduled on an unavailable field date.")
                
                # Check for partial-day unavailability
                for part_day in field_info.get('part_days', []):
                    if game_date == part_day.date() and game.timeslot.time == part_day.time():
                        violations.append(f"Game {game} was scheduled during an unavailable time slot.")
    
    return violations

def check_preference_violations(roster, data):
    """Check if any games in the roster violate the preference_no_play constraints."""
    violations = []
    preference_no_play = data['preference_no_play']

    for club, restrictions in preference_no_play.items():
        for restriction in restrictions:
            club_teams = get_teams_from_club(club, data['teams'])
            specific_team = restriction.get('team_name')

            if specific_team:
                assert specific_team in club_teams, f"Team {specific_team} not found in club {club}."
                for week in roster.weeks:
                    for game in week.games:
                        if specific_team in [game.team1, game.team2]:
                            if all(getattr(game.timeslot, key, None) == value or getattr(game.field, key, None) == value 
                                    for key, value in restriction.items()):
                                violations.append(f"Game {game} violates preference_no_play for club {club}.")
            else:
                for week in roster.weeks:
                    for game in week.games:
                        if game.team1 in club_teams or game.team2 in club_teams:

                            if all(getattr(game.timeslot, key, None) == value or getattr(game.field, key, None) == value 
                                    for key, value in restriction.items()):
                                violations.append(f"Game {game} violates preference_no_play for club {club}.")
        
        # Check for team conflict violations
        for (team1, team2) in data['team_conflicts']:
            for week in roster.weeks:
                scheduled_games = []
                for game in week.games:
                    if game.team1 in [team1, team2] or game.team2 in [team1, team2]:
                        game_time = (game.timeslot.date, game.timeslot.time, game.field.location)
                        if game_time in scheduled_games:
                            violations.append(f"Teams {team1} and {team2} were scheduled to play at the same time: {game}.")
                        scheduled_games.append(game_time)
    
    return violations

roster = convert_X_to_roster(X, data)
# export_roster_to_excel(roster)

ValidationError: 2 validation errors for Game
field
  Input should be a valid dictionary or instance of PlayingField [type=model_type, input_value=PlayingField(name='SF', l...national Hockey Centre'), input_type=PlayingField]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
grade
  Input should be a valid dictionary or instance of Grade [type=model_type, input_value=Grade(name='5th', teams=[...ts  5th'], num_teams=10), input_type=Grade]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

In [ ]:
def analyze_roster(roster: Roster, data: dict):
    field_usage = defaultdict(set)  # {date_time: field}
    matchup_matrices = {}
    game_counts_per_week = defaultdict(lambda: defaultdict(set))
    violations = []
    
    # Iterate through weekly draws
    for weekly_draw in roster.weeks:
        week_num = weekly_draw.week
        for game in weekly_draw.games:
            grade_name = game.grade.name
            team1, team2 = game.team1, game.team2
            timeslot_key = (game.timeslot.date, game.timeslot.time, game.field.name)

            # ✅ 1. Field Conflict Check                                                                                                NoDoubleBookingFieldsConstraint()
            game_identifier = (game.team1, game.team2, game.grade.name)
            if timeslot_key in field_usage:
                violations.append(f"Field Conflict: {game.field} has multiple games at {game.timeslot.date} {game.timeslot.time}.")
            field_usage[timeslot_key].add(game_identifier)

            # ✅ 2. Matchup Matrix
            if grade_name not in matchup_matrices:
                teams = game.grade.teams
                matchup_matrices[grade_name] = pd.DataFrame(0, index=teams, columns=teams)
            
            matchup_matrices[grade_name].loc[team1, team2] += 1
            matchup_matrices[grade_name].loc[team2, team1] += 1
            game_counts_per_week[grade_name][team1].add(week_num)
            game_counts_per_week[grade_name][team2].add(week_num)
    
# ✅ 3. Balanced Participation Check                                                                                                        EnsureEqualGamesAndBalanceConstraint()
    for grade, matrix in matchup_matrices.items():
        teams = list(matrix.index)
        games_played = {team: sum(matrix.loc[team]) for team in teams}
        matchups = matrix.to_dict()

        total_weeks = set(range(1, len(roster.weeks) + 1))
        byes = {team: len(total_weeks) - len(game_counts_per_week[grade][team]) for team in teams}
        
        if len(set([len(game_counts_per_week[grade][team]) for team in teams])) > 1:
            violations.append(f"Different measure, unequal games in {grade}: {games_played} \n {matrix}")
        if len(set(games_played.values())) > 1:
            violations.append(f"Unequal games in {grade}: {games_played} \n {matrix}")
        if len(set(byes.values())) > 1:
            violations.append(f"Unequal byes in {grade}: {byes} \n {matrix}")
        for team, opponents in matchups.items():
            del opponents[team]
            if len(set(opponents.values())) > 1:
                violations.append(f"Unequal matchups in {grade}: {team} has {opponents} \n {matrix}")
    
    # ✅ 4. PHL Time Restrictions                                                                                                               
    for weekly_draw in roster.weeks:
        phl_dict = defaultdict(list)
        for game in weekly_draw.games:
            if game.grade.name == 'PHL' and game.field.location == 'Newcastle International Hockey Centre':
                phl_dict[(game.timeslot.date, game.timeslot.day_slot)].append(game)
                if (int(game.timeslot.time[:2]) < 11 or int(game.timeslot.time[:2]) >= 16):
                    violations.append(f"PHL game outside allowed hours: {game.timeslot.date} at {game.timeslot.time} on {game.field.location}.")
        for games in phl_dict.values():
            if len(games) > 1:
                violations.append(f"PHL games on same timeslot: {', '.join(game.team1 for game in games)} on {games[0].timeslot.date}.")
        # PHL and Second grade in same club time restriction
    teams = data['teams']
    for weekly_draw in roster.weeks:
        phl_dict = defaultdict(lambda: defaultdict(list))
        for game in weekly_draw.games:
            if game.grade.name in ['PHL', '2nd']:
                phl_dict[(game.timeslot.date, game.timeslot.time)][get_club(game.team1, teams)].append(game)
                phl_dict[(game.timeslot.date, game.timeslot.time)][get_club(game.team2, teams)].append(game)
        for date, clubs in phl_dict.items():
            for club, games in clubs.items():
                if len(games) > 1 and games[0] != games[1] and len(set(game.grade.name for game in games)) > 1:
                    teams_ = {game.team1 for game in games if club in game.team1} | {game.team2 for game in games if club in game.team2}
                    violations.append(f"PHL and 2nd grade timing violation: {', '.join(teams_)} on {games[0].timeslot.date}.")
                    
    # ✅ 5. Club Scheduling Rule for 2nds and PHL
    phl_games_by_week_field = defaultdict(lambda: defaultdict(list))  # {week: {field: [day_slots]}}
    second_grade_games_by_week_field = defaultdict(lambda: defaultdict(list))
    
    for weekly_draw in roster.weeks:
        for game in weekly_draw.games:
            if game.grade.name == 'PHL':
                phl_games_by_week_field[weekly_draw.week,game.timeslot.day, get_club(game.team1, teams)][game.field.location].append(game.timeslot.time)
                phl_games_by_week_field[weekly_draw.week, game.timeslot.day, get_club(game.team2, teams)][game.field.location].append(game.timeslot.time)

            elif game.grade.name == '2nd':
                second_grade_games_by_week_field[weekly_draw.week, game.timeslot.day, get_club(game.team1, teams)][game.field.location].append(game.timeslot.time)
                second_grade_games_by_week_field[weekly_draw.week, game.timeslot.day, get_club(game.team2, teams)][game.field.location].append(game.timeslot.time)

    
    for week, fields in phl_games_by_week_field.items():            
        for field, phl_slots in fields.items():
            second_slots = second_grade_games_by_week_field[week]
            for second_field, second_slot in second_slots.items():
                datetime.strptime(slot) - timedelta(minutes=120)
                if second_field != field and any(slot - 1 in second_slots or slot + 1 in second_slots for slot in phl_slots):
                    violations.append(f"PHL/2nd scheduling violation in week {week} with 2nds on field {second_field} at time {second_slot} and PHL on {field} at time {phl_slots}.")
    
    # ✅ 6. Game Count Per Grade
    grade_counts = defaultdict(lambda: defaultdict(int))
    for weekly_draw in roster.weeks:
        for game in weekly_draw.games:
            grade_counts[game.grade.name][game.grade.num_teams] += 1 

    # ✅ 7. Soft Constraint Violations
    if 'penalties' in data:
        for constraint_name, constraint in data['penalties'].items():
            for penalty_var in constraint['unresolved']:
                violations.append(f"{constraint_name} violation: {penalty_var}")
    # 8. Back to Back Byes
    potential_violations = check_back_to_back_byes(roster)
    for week, teams in potential_violations.items():
        violations.append(f"Teams {', '.join(teams)} have consecutive byes with second bye in week {week}.")

    # 9. Check Maitland home/away proportions
    potential_violations = check_home_proportion(roster, data)
    for team, proportion in potential_violations.items():
        violations.append(f"Team {team} has {proportion} of games at home.")
        
    # 10. Check complex Maitland requirements
    potential_violations = check_maitland_complex_limitations(roster, data)
    for key, value in potential_violations.items():
        violations.append(f"{key}: {value}")
        
    # 11. Check no draw gaps
    potential_violations = check_no_draw_gaps(roster, data)
    for week, fields in potential_violations.items():
        violations.append(f"Draw gap in week {week} at field {fields[0]}: {fields[0]}")
        
    # 12. Check for requested unavailability
    potential_violations = check_roster_for_requested_unavailability(roster, data['unavailable_games'], data['field_unavailabilities'])
    for viol in potential_violations:
        violations.append(viol)
        
    # 13. Check for preference violations
    potential_violations = check_preference_violations(roster, data)
    for viol in potential_violations:
        violations.append(viol)
        
    # ✅ Print Results
    print("Assessed Conditions:")
    if violations:
        for violation in violations:
            print(violation)
    else:
        print("No violations found.")
    
    print("\nNumber of Games Per Grade:")
    for grade, num_teams in grade_counts.items():
        print(f"{grade}: {2 * list(num_teams.values())[0]/list(num_teams.keys())[0]}") # Multiply by 2 because there are 2 teams per game count

# Example Usage
analyze_roster(roster, data)

(1, 'Sunday', 'Norths')
defaultdict(<class 'list'>, {})
Assessed Conditions:
PHL game outside allowed hours: 2025-03-23 at 16:00 on Newcastle International Hockey Centre.
PHL game outside allowed hours: 2025-04-06 at 16:00 on Newcastle International Hockey Centre.
PHL game outside allowed hours: 2025-05-11 at 16:00 on Newcastle International Hockey Centre.
PHL game outside allowed hours: 2025-05-25 at 16:00 on Newcastle International Hockey Centre.
PHL game outside allowed hours: 2025-06-15 at 16:00 on Newcastle International Hockey Centre.
PHL game outside allowed hours: 2025-06-22 at 16:00 on Newcastle International Hockey Centre.
PHL game outside allowed hours: 2025-06-29 at 16:00 on Newcastle International Hockey Centre.
PHL game outside allowed hours: 2025-07-13 at 16:00 on Newcastle International Hockey Centre.
PHL game outside allowed hours: 2025-07-27 at 16:00 on Newcastle International Hockey Centre.
PHL game outside allowed hours: 2025-08-10 at 16:00 on Newcastle Internationa

In [ ]:
import csv
from datetime import datetime

def enforce_schedule_from_csv(model, X, csv_filename):
    """
    Reads a CSV file and updates the constraint model by enforcing 
    the correct game variables to be set based on the schedule.
    """
    scheduled_games = set()  # To store games from the CSV for validation
    
    # Step 1: Read CSV file
    with open(csv_filename, newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter='\t')  # Assuming tab-separated values
        for row in reader:
            team1 = row['Team 1'].strip()
            team2 = row['Team 2'].strip()
            grade = row['Grade'].strip()
            field_name = row['Field Name'].strip()
            date = row['Date'].strip()
            day = row['Day'].strip()
            time = row['Time'].strip()

            # Convert date and time to correct formats
            game_date = datetime.strptime(date, "%Y-%m-%d").date()
            game_time = datetime.strptime(time, "%H:%M").time()

            # Step 2: Find the matching game variable in X (accounting for Team 1 and Team 2 being interchangeable)
            matching_keys = [
                key for key in X
                if (
                    ((key[0] == team1 and key[1] == team2) or (key[0] == team2 and key[1] == team1))  # Match teams in either order
                    and key[2] == grade  # Match grade
                    and key[3] == day  # Match day
                    and key[4] == game_time  # Match time
                    and key[5] == game_date  # Match date
                    and key[9] == field_name  # Match field name
                    and key[7] == 1  # Ensure game is in Round 1
                )
            ]

            assert len(matching_keys) == 1, f"Error: Expected one match for game {team1} vs {team2}, found {len(matching_keys)}"

            # Step 3: Set the matched game variable to 1
            game_key = matching_keys[0]
            model.Add(X[game_key] == 1)

            # Store scheduled games for validation
            scheduled_games.add(game_key)

    # Step 4: Set all other games in Round 1 to 0
    for key in X:
        if key[7] == 1 and key not in scheduled_games:  # Only consider Round 1 games
            model.Add(X[key] == 0)

    # Step 5: Validation - Ensure every game in X has at most one match in the schedule
    assert len(scheduled_games) == len(set(scheduled_games)), "Error: Duplicate games detected in the schedule."

    print(f"Successfully enforced {len(scheduled_games)} scheduled games from {csv_filename}.")


[Game(team1='Colts 4th', team2='Maitland 4th', timeslot=Timeslot(date='2025-03-30', day='Sunday', time='13:00', week=1, day_slot=2, field=PlayingField(name='South Field', location='Broadmeadow')), field=PlayingField(name='South Field', location='Broadmeadow'), grade=Grade(name='4th', teams=['Colts 4th', 'Maitland 4th', 'Norths 4th', 'Port Stephens 4th', 'Souths 4th', 'Tigers 4th', 'University 4th', 'Wests 1 4th', 'Wests 2 4th'], num_teams=9)),
 Game(team1='Maitland 5th', team2='Norths 5th', timeslot=Timeslot(date='2025-03-30', day='Sunday', time='13:00', week=1, day_slot=2, field=PlayingField(name='Maitland Main Field', location='Maitland')), field=PlayingField(name='Maitland Main Field', location='Maitland'), grade=Grade(name='5th', teams=['Colts 5th', 'Crusaders 5th', 'Maitland 5th', 'Norths 5th', 'Port Stephens 5th', 'Tigers 1 5th', 'Tigers 2 5th', 'University 1 5th', 'University 2 5th', 'Wests  5th'], num_teams=10)),
 Game(team1='Souths 2nd', team2='Tigers 2nd', timeslot=Timeslot(d

In [ ]:
with open(r"C:\Users\c3205\Documents\Code\Jupyter Notebooks\backup_draw\hockey_draw\timeslots_working.json", "r") as file:
    loaded_timeslots = json.load(file)

# Convert field dictionary back to PlayingField object
for item in loaded_timeslots:
    item['field'] = PlayingField(**item['field'])
class Timeslot1(BaseModel):
    date: str = Field(..., description="Date of the game (e.g., '2025-03-04')")
    day: str = Field(..., description="Day of the game (e.g., 'Saturday', 'Sunday')")
    time: str = Field(..., description="Time of the game (e.g., 14:00 for 2 PM)")
    week: int = Field(..., description="The week number for the season")
    day_slot: int = Field(..., description="The game slot for the day (e.g., 1 for first game of the day)")
    field: PlayingField = Field(..., description="Field where the game is played")


LOADEDTIMESLOTS = [Timeslot1(date=t['date'], day=t['day'], time=t['time'], week=t['week'], day_slot=t['day_slot'], field=t['field']) for t in loaded_timeslots]


In [44]:
import pickle
with open(r"C:\Users\c3205\Documents\Code\Jupyter Notebooks\draw\draws\V1\X.pkl", "rb") as f:
    loaded_X = pickle.load(f)

with open(r"C:\Users\c3205\Documents\Code\Jupyter Notebooks\draw\draws\V1\data.pkl", "rb") as f:
    loaded_data = pickle.load(f)
roster = convert_X_to_roster(loaded_X, data)
analyze_roster(roster, data)



ValidationError: 2 validation errors for Game
field
  Input should be a valid dictionary or instance of PlayingField [type=model_type, input_value=PlayingField(name='South ... location='Broadmeadow'), input_type=PlayingField]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
grade
  Input should be a valid dictionary or instance of Grade [type=model_type, input_value=Grade(name='5th', teams=[...ts  5th'], num_teams=10), input_type=Grade]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type

In [21]:
import pandas as pd
from typing import Dict
from pydantic import BaseModel
from pathlib import Path

def load_roster_from_excel(filename: str, data: Dict) -> Roster:
    all_teams: Set[str] = {team.name for team in data['teams']}  # Extract all team names
    roster_weeks = []

    # Load Excel file
    xls = pd.ExcelFile(filename)

    for sheet_name in xls.sheet_names:
        week = int(sheet_name.replace("Week ", "").strip())  # Extract week number
        df = pd.read_excel(xls, sheet_name=sheet_name)

        games = []
        teams_played = set()

        for _, row in df.iterrows():
            field_class = get_field_by_name(row["Field Name"].split("' location")[0].split("'")[1], data['fields'])
            timeslot = Timeslot(day=row["Day"], time=row["Time"], week=week, field=field_class, day_slot=None, date=None)

            game = Game(
                team1=row["Team 1"],
                team2=row["Team 2"],
                timeslot=timeslot,
                field=field_class,
                grade=None  # Grade is missing from the Excel file, needs handling
            )

            games.append(game)
            teams_played.update([row["Team 1"], row["Team 2"]])

        # Identify bye teams
        bye_teams = list(all_teams - teams_played)

        # Create WeeklyDraw
        weekly_draw = WeeklyDraw(week=week, games=games, bye_teams=bye_teams)
        roster_weeks.append(weekly_draw)

    return Roster(weeks=roster_weeks)

roster_File = r'C:\Users\c3205\Documents\Code\Jupyter Notebooks\draw\draws\V1\schedule.xlsx'

roster = load_roster_from_excel(roster_File, data)

KeyboardInterrupt: 

In [10]:
FIELDS

[PlayingField(name='South Field', location='Broadmeadow'),
 PlayingField(name='East Field', location='Broadmeadow'),
 PlayingField(name='West Field', location='Broadmeadow'),
 PlayingField(name='Maitland Main Field', location='Maitland'),
 PlayingField(name='Wyong Main Field', location='Wyong')]